# イベント画像を OpenAI で解析する

---

## Guide page
このサンプルを実行する場合は、必ず以下の**ガイドページ**を確認してください。
- https://users.soracom.io/ja-jp/docs/soracom-cloud-camera-services/api-examples-analyze-event-image-with-openai/


## License

We are using the following libraries.
- **openai / openai-python**
  - https://github.com/openai/openai-python

The license complies with the library's license.
- **Apache License 2.0**
  - https://github.com/openai/openai-python/blob/main/LICENSE

In [ ]:
# @title ステップ 1: OpenAI のライブラリをインストールする

#
# --------------- Execution ---------------
#

# When executing 'pip install', an error is displayed due to the absence of the following dependent libraries, but it is commented out as it does not affect the execution.
# !pip install --upgrade cohere
# !pip install --upgrade tiktoken

!pip install --upgrade openai

In [ ]:
# @title ステップ 2: 定数の設定と共通ライブラリをインポートする

"""
Perform common processing and define constants.

Note:
    - Define common functions used in sample code.
    - Define constants used in sample code.
    - All samples use common code.
    - Some elements are not used directly in certain samples.
    - The Python docstrings are generated using ChatGPT3.5.
"""

#
# --------------- Definition ---------------
#

import filecmp
import glob
import json
import os
import shutil
import sys
from datetime import datetime, timedelta, timezone
from enum import Enum
from typing import Final
from urllib.parse import quote, unquote, urljoin, urlparse

import matplotlib.pyplot as plt
import requests
from google.colab import files
from mpl_toolkits.axes_grid1 import ImageGrid
from PIL import Image


class APIResult(object):
    """
    Class to Hold the Results of API Execution

    This class is used to encapsulate the results of an API execution, including the API response and additional information.

    Attributes:
        response (dict): The API response data.
        additions (dict): Additional information related to the API response.

    Methods:
        __init__(response: dict, additions: dict): Constructor for initializing the APIResult object.
        get_display_response(): Retrieve and format the API response data for display.
        get_display_additions(): Retrieve and format additional information for display.
        check_success(): Check if the API execution was successful based on the HTTP status code.
    """

    def __init_vars(self):
        """
        Initialize instance variables.

        This method initializes the 'response' and 'additions' attributes.
        """
        self.response = None
        self.additions = None

    def __init__(self, response: dict, additions: dict):
        """
        Initialize an APIResult object.

        Args:
            response (dict): The API response data.
            additions (dict): Additional information related to the API response.
        """
        self.__init_vars()
        self.response = response
        self.additions = additions

    def get_display_response(self) -> str:
        """
        Retrieve and format the API response data for display.

        Returns:
            str: Formatted API response data for display. Returns an empty string if there is no response.
        """
        if self.response is not None:
            return json.dumps(self.response, indent=4, ensure_ascii=False)
        return ''

    def get_display_additions(self) -> str:
        """
        Retrieve and format additional information for display.

        Returns:
            str: Formatted additional information for display. Returns an empty string if there is no additional information.
        """
        if self.additions is not None:
            return json.dumps(self.additions, indent=4, ensure_ascii=False)
        return ''

    def check_success(self) -> bool:
        """
        Check if the API execution was successful based on the HTTP status code.

        Returns:
            bool: True if the HTTP status code in the additional information is 200 or 204, otherwise False.
        """
        if self.additions is not None:
            return self.additions.get('status_code') in [200, 204]
        return False


class ContentsResult(object):
    """
    Class to Hold the Results of Content Execution

    This class is used to store the results of content execution, including the content itself
    and additional information.

    Attributes:
        content (bytes): The content returned from the content request.
        additions (dict): Additional information related to the content request.

    Methods:
        __init__(content: bytes, additions: dict): Constructor for initializing the object.
        get_display_additions(): Retrieve formatted additional information for display.
        check_success(): Check if the content execution was successful.

    """

    def __init_vars(self):
        """
        Initialize instance variables.

        This method initializes the 'content' and 'additions' attributes.
        """
        self.content = None
        self.additions = None

    def __init__(self, content: bytes, additions: dict):
        """
        Initialize a ContentsResult object.

        Args:
            content (bytes): The content returned from the content request.
            additions (dict): Additional information related to the content request.
        """
        self.__init_vars()
        self.content = content
        self.additions = additions

    def get_display_additions(self) -> str:
        """
        Retrieve formatted additional information for display.

        Returns:
            str: Formatted additional information for display. Returns an empty string if there is no response.
        """
        if self.additions is not None:
            return json.dumps(self.additions, indent=4, ensure_ascii=False)
        return ''

    def check_success(self) -> bool:
        """
        Check if the content execution was successful.

        Returns:
            bool: True if the HTTP status code in the additional information is 200, otherwise False.
        """
        if self.additions is not None:
            return True if self.additions.get('status_code') == 200 else False
        return False


class ExportTimeInfo(object):
    """
    Class representing time intervals for data export.

    This class is used to manage and validate time intervals for exporting data.

    Attributes:
        time_from (datetime): The start time of the export interval.
        time_to (datetime): The end time of the export interval.

    Methods:
        valid_vars(): Check if the instance variables are valid.
        get_from_ms(): Return milliseconds of the start datetime.
        get_to_ms(): Return milliseconds of the end datetime.
        get_export_sec(): Calculate the duration of the export time.
        check_time_limits(range_limit=True): Check if the set time limits are valid.
        create_time_list(): Create a list of ExportTimeInfo objects representing time intervals for data export.
    """

    def init_vars(self):
        """Initialize instance variables."""
        self.time_from = None
        self.time_to = None

    def __init__(self, time_from: datetime, time_to: datetime):
        """
        Initialize an ExportTimeInfo object.

        Args:
            time_from (datetime): The start time of the export interval.
            time_to (datetime): The end time of the export interval.
        """
        self.init_vars()
        self.time_from = time_from
        self.time_to = time_to

    def valid_vars(self) -> bool:
        """
        Check if instance variables are valid.

        Returns:
            bool: True if both instance variables are valid, False otherwise.
        """
        return True if self.time_from is not None and self.time_to is not None else False

    def __get_range_limit(self) -> int:
        """
        Get the maximum duration for exporting in seconds.

        Returns:
            int: The maximum duration in seconds for exporting.
        """
        # As of June 2023, there is a limit of 15 minutes per video export.
        # Reference: https://users.soracom.io/ja-jp/tools/api/reference/#/SoraCam/getSoraCamDeviceStreamingVideo
        EXPORT_RANGE_LIMIT_SECONDS: Final = 900
        return EXPORT_RANGE_LIMIT_SECONDS

    def get_from_ms(self) -> int | None:
        """
        Get milliseconds of the start datetime.

        Returns:
            int: Milliseconds of the start datetime.
            None: None if the start datetime is invalid.
        """
        if self.time_from is not None:
            return int(self.time_from.timestamp() * 1000)
        return None

    def get_to_ms(self) -> int | None:
        """
        Get milliseconds of the end datetime.

        Returns:
            int: Milliseconds of the end datetime.
            None: None if the end datetime is invalid.
        """
        if self.time_to is not None:
            return int(self.time_to.timestamp() * 1000)
        return None

    def get_export_sec(self) -> int | None:
        """
        Calculate the duration of the export time in seconds.

        Returns:
            int: The duration of the export time in seconds.
            None: None if the start and end datetime are invalid.
        """
        if self.valid_vars():
            return int((self.time_to - self.time_from).total_seconds())
        return None

    def check_time_limits(self, range_limit: bool = True) -> bool:
        """
        Check the validity of the time limits.

        Args:
            range_limit (bool, optional): Specify True to include the API export limit.
                                          The default value is True.

        Returns:
            bool: True if there are no issues with the check and the value is available.
        """
        if not self.valid_vars():
            return False

        jst_now = datetime.now(JST)

        # Future time cannot be specified.
        if jst_now.timestamp() < self.time_from.timestamp() or jst_now.timestamp() < self.time_to.timestamp():
            print()
            print(
                '➕➕➕ ⛔ A future date and time has been specified. Please confirm the date and time again. ➕➕➕')
            return False

        # If there is no difference between start and end time
        if self.get_export_sec() <= 0:
            print()
            print(
                '➕➕➕ ⛔ The date and time specified are incorrect. Please check the date and time specified. ➕➕➕')
            return False

        # Factoring in API unit output limits
        if range_limit:
            if self.get_export_sec() > self.__get_range_limit():
                print()
                print('➕➕➕ ⛔ The specified date and time range exceeds {} seconds. Please specify within the limit. ➕➕➕'.format(
                    self.__get_range_limit()))
                return False

        return True

    def create_time_list(self) -> list:
        """
        Create a list of ExportTimeInfo objects representing time intervals for data export.

        Returns:
            list: A list of ExportTimeInfo objects, each representing a time interval.
        """
        time_list = list()

        if not self.valid_vars():
            return time_list

        start = self.time_from
        end = self.time_to

        if not self.check_time_limits(False):
            return time_list

        # Use as is if not caught by the output limit of the export unit.
        if (end - start).total_seconds() <= self.__get_range_limit():
            time_list.append(ExportTimeInfo(start, end))
            return time_list

        # Divide by output unit.
        while True:
            # Add Output Units Only
            end = start + timedelta(seconds=self.__get_range_limit())

            # The calculated time is in the future and should be the specified time.
            if (self.time_to - end).total_seconds() <= 0:
                time_list.append(ExportTimeInfo(start, self.time_to))
                break

            time_list.append(ExportTimeInfo(start, end))
            start = end

        return time_list


class EnumIntervalUnit(Enum):
    """
    Enumeration representing different time interval units.

    Attributes:
        MINUTE (str): Represents a time interval of one minute.
        HOUR (str): Represents a time interval of one hour.
        DAY (str): Represents a time interval of one day.
        WEEK (str): Represents a time interval of one week.

    Methods:
        get_all_names(): Returns a list of all enumeration names.
        get_all_values(): Returns a list of all enumeration values.
    """
    MINUTE = 'Minute'
    HOUR = 'Hour'
    DAY = 'Day'
    WEEK = 'Week'

    @classmethod
    def get_all_names(cls) -> list:
        """
        Get a list of all enumeration names.

        Returns:
            list: A list of strings representing enumeration names.
        """
        return [i.name for i in cls]

    @classmethod
    def get_all_values(cls) -> list:
        """
        Get a list of all enumeration values.

        Returns:
            list: A list of strings representing enumeration values.
        """
        return [i.value for i in cls]


class ExportImageTimeInfo(object):
    """
    Class for Managing Export Image Time Intervals

    This class is used to manage and validate time intervals for exporting images.

    Attributes:
        time_from (datetime): The start time for exporting images.
        interval (float): The time interval between image exports.
        interval_unit (str): The unit of time for the interval (e.g., 'Minute', 'Hour', 'Day', 'Week').
        fetch_count (int): The number of image exports to be fetched.

    Methods:
        init_vars(): Initialize instance variables.
        valid_vars() -> bool: Check if instance variables are valid.
        __init__(time_from: datetime, interval: float, interval_unit: str, fetch_count: int): Constructor for initializing the object.
        check_time_status(range_limit=True) -> bool: Check the validity of time intervals and attributes.
        create_time_list() -> list: Create a list of export times based on the specified interval and count.

    """

    def init_vars(self):
        """
        Initialize instance variables.

        This method initializes the 'time_from', 'interval', 'interval_unit', and 'fetch_count' attributes.
        """
        self.time_from = None
        self.interval = 0.0
        self.interval_unit = None
        self.fetch_count = 0

    def valid_vars(self) -> bool:
        """
        Check if instance variables are valid.

        Returns:
            bool: True if all instance variables are valid, otherwise False.
        """
        return (
            self.time_from is not None and
            self.interval != 0 and
            self.interval_unit is not None and
            self.interval_unit != '' and
            self.fetch_count >= 2
        )

    def __init__(self, time_from: datetime = None, interval: float = 0.0, interval_unit: str = None, fetch_count: int = 0):
        """
        Initialize an ExportImageTimeInfo object.

        Args:
            time_from (datetime): The start time for exporting images.
            interval (float): The time interval between image exports.
            interval_unit (str): The unit of time for the interval (e.g., 'Minute', 'Hour', 'Day', 'Week').
            fetch_count (int): The number of image exports to be fetched.
        """
        self.init_vars()
        self.time_from = time_from
        self.interval = interval
        self.interval_unit = interval_unit
        self.fetch_count = fetch_count

    def check_time_status(self, range_limit: bool = True) -> bool:
        """
        Check the validity of time intervals and attributes.

        Args:
            range_limit (bool, optional): Specify True to include the API export limit. The default is True.

        Returns:
            bool: True if the time intervals and attributes are valid, otherwise False.
        """
        jst_now = datetime.now(JST)

        # Future time cannot be specified.
        if jst_now.timestamp() < self.time_from.timestamp():
            print()
            print(
                '➕➕➕ ⛔ A future date and time has been specified. Please confirm the date and time again. ➕➕➕')
            return False

        # Zero interval is not allowed.
        if self.interval == 0:
            print()
            print(
                '➕➕➕ ⛔ Zero interval is not allowed. Please confirm. ➕➕➕')
            return False

        # Are all of the values valid?
        if not self.valid_vars():
            print()
            print(
                '➕➕➕ ⛔ The entered values include some that are invalid. Please check. ➕➕➕')
            return False

        # Not allowed if future dates are included in the acquisition list.
        time_list = self.create_time_list()
        if len(time_list) > 0:
            latest_time = time_list[-1]
            if jst_now.timestamp() < latest_time.timestamp():
                print()
                print(
                    '➕➕➕ ⛔ Future dates are included in the acquisition list. Please check. ➕➕➕')
                return False

        return True

    def create_time_list(self) -> list:
        """
        Create a list of export times based on the specified interval and count.

        Returns:
            list: A list of datetime objects representing export times.
        """
        time_list = list()

        if not self.valid_vars():
            return time_list

        # Create a list of acquisition times according to the interval and number of copies to be acquired at each time from the start date.
        time_list.append(self.time_from)
        counter = 1
        while counter < self.fetch_count:
            if self.interval_unit == EnumIntervalUnit.MINUTE.value:
                time_list.append(
                    time_list[counter-1] + timedelta(minutes=self.interval))
            elif self.interval_unit == EnumIntervalUnit.HOUR.value:
                time_list.append(
                    time_list[counter-1] + timedelta(hours=self.interval))
            elif self.interval_unit == EnumIntervalUnit.DAY.value:
                time_list.append(
                    time_list[counter-1] + timedelta(days=self.interval))
            elif self.interval_unit == EnumIntervalUnit.WEEK.value:
                time_list.append(
                    time_list[counter-1] + timedelta(weeks=self.interval))

            counter += 1

        time_list.sort()

        return time_list


#
# --------------- Definition of constants ---------------
#


# Set Japan time zone
JST: Final = timezone(timedelta(hours=+9), 'Asia/Tokyo')

# Working directory when creating a ZIP file
ZIP_WORK_DIR: Final = './__zip_work'

# The home directory for user content.
USER_CONTENT_DIR: Final = '/content/'

# Define where to store downloaded files
EVENT_IMAGE_DIR: Final = './event_image'

# Define where to store downloaded files
EXPORT_IMAGES_DIR: Final = './export_images'


def print_start_msg() -> None:
    """
    Display the start message for processing.

    This function displays a start message along with the current date and time in the Japan Standard Time (JST) zone.
    """
    print()
    print('# ===== 🚩 Start processing =====', datetime.now(JST))


def print_end_msg() -> None:
    """
    Display the end message for processing.

    This function displays an end message along with the current date and time in the Japan Standard Time (JST) zone.
    """
    print()
    print('# ===== 🍵 End processing =====', datetime.now(JST))


def merge_url(base: str, url: str) -> str:
    """
    Combine URL Strings.

    Args:
        base (str): The base URL.
        url (str): The URL to be combined with the base URL.

    Returns:
        str: The combined URL.

    This function combines a base URL and a relative URL to generate a complete URL. It ensures that the resulting URL is properly formatted.
    """
    print()
    print('# 🧰 Combine URL Strings')
    print('# base = ', base)
    print('# url = ', url)

    parsed = urlparse(base)
    if parsed.path != '':
        if not parsed.path.endswith('/'):
            # add '/' because it is missing at the end
            base += '/'

    return urljoin(base, url.replace('/', '', 1))


def send_post_request(url: str, headers: dict, data: dict, log: bool = False) -> APIResult:
    """
    Send an HTTP POST request to the specified URL with the given headers and data.

    Args:
        url (str): The URL to which the POST request will be sent.
        headers (dict): A dictionary containing the HTTP headers to include in the request.
        data (dict): A dictionary containing the data to be included in the request body.
        log (bool, optional): Whether to log the headers and data before sending the request. Default is False.

    Returns:
        APIResult: An APIResult object containing the result of the HTTP POST request.

    Raises:
        (Any exceptions that this function may raise)

    Note:
        This function sends an HTTP POST request to the specified URL using the requests library.
        If the response status code is 200, it returns the JSON response along with additional information.
        If the response status code is 204, it returns None along with additional information.
        If the request fails, it logs the error and returns the error response JSON along with additional information.
    """
    print()
    print('# 📡 Send HTTP POST request')
    print('# url = ', url)

    if log:
        print('# headers = ', headers)
        print('# data = ', data)

    # Send Request
    resp = requests.post(url, headers=headers, data=json.dumps(data))

    # Display request datetime in JST
    # Mon, 06 Feb 2023 02:03:09 GMT
    req_time = datetime.strptime(
        resp.headers['Date'], '%a, %d %b %Y %H:%M:%S %Z').astimezone(JST)
    print('# ⏰ Request datetime = ', req_time,
          int(req_time.timestamp() * 1000))

    # Preparation for results return
    additions = dict()
    additions['status_code'] = resp.status_code
    additions['headers'] = resp.headers

    if resp.status_code == 200:
        return APIResult(resp.json(), additions)

    if resp.status_code == 204:
        return APIResult(None, additions)

    print()
    print('# ❌ POST request failed')
    print('# Status Code = ', resp.status_code)
    print('# Raw Messages = ', resp.json())
    additions['raw_messages'] = resp.json

    return APIResult(resp.json(), additions)


def send_get_request(url: str, headers: dict, params: dict = None, log: bool = False) -> APIResult:
    """
    Send an HTTP GET request to the specified URL with the given headers and optional parameters.

    Args:
        url (str): The URL to which the GET request will be sent.
        headers (dict): A dictionary containing the HTTP headers to include in the request.
        params (dict, optional): A dictionary containing query parameters to include in the request. Default is None.
        log (bool, optional): Whether to log the headers and parameters before sending the request. Default is False.

    Returns:
        APIResult: An APIResult object containing the result of the HTTP GET request.

    Note:
        This function sends an HTTP GET request to the specified URL using the requests library.
        If the response status code is not 200, it logs the error and returns the error response JSON along with additional information.
    """
    print()
    print('# 📡 Send HTTP GET request')
    print('# url = ', url)

    if log:
        print('# headers = ', headers)
    if params is not None:
        print('# params = ', params)

    # Send Request
    resp = requests.get(url, headers=headers, params=params)

    # Display request datetime in JST
    # Mon, 06 Feb 2023 02:03:09 GMT
    req_time = datetime.strptime(
        resp.headers['Date'], '%a, %d %b %Y %H:%M:%S %Z').astimezone(JST)
    print('# ⏰ Request datetime = ', req_time,
          int(req_time.timestamp() * 1000))

    # Preparation for results return
    additions = dict()
    additions['status_code'] = resp.status_code
    additions['headers'] = resp.headers

    if resp.status_code != 200:
        print()
        print('# ❌ GET request failed')
        print('# Status Code = ', resp.status_code)
        print('# Raw Messages = ', resp.json())
        additions['raw_messages'] = resp.json

    return APIResult(resp.json(), additions)


def send_get_content(url: str) -> ContentsResult:
    """
    Retrieve content via an HTTP GET request from the specified URL.

    Args:
        url (str): The URL from which content will be retrieved.

    Returns:
        ContentsResult: A ContentsResult object containing the retrieved content and additional information.

    Note:
        This function sends an HTTP GET request to the specified URL using the requests library.
        If the response status code is not 200, it logs the error and returns the error response content along with additional information.
    """
    print()
    print('# 💾 Retrieve contents')
    print('# url = ', url)

    # Send Request
    resp = requests.get(url)

    # Display request datetime in JST
    # Mon, 06 Feb 2023 02:03:09 GMT
    req_time = datetime.strptime(
        resp.headers['Date'], '%a, %d %b %Y %H:%M:%S %Z').astimezone(JST)
    print('# ⏰ Request datetime = ', req_time,
          int(req_time.timestamp() * 1000))

    # Preparation for results return
    additions = dict()
    additions['status_code'] = resp.status_code
    additions['headers'] = resp.headers

    if resp.status_code != 200:
        print()
        print('# ❌ GET contents failed')
        print('# Status Code = ', resp.status_code)
        print('# Raw Messages = ', resp.json())
        additions['raw_messages'] = resp.json

    return ContentsResult(resp.content, additions)


def extract_filename_from_URL(url: str) -> str:
    """
    Extract the filename from a URL string.

    Args:
        url (str): The URL from which the filename will be extracted.

    Returns:
        str: The extracted filename, URL-decoded and then URL-encoded.

    Note:
        This function takes a URL as input and extracts the filename from it.
        The extracted filename is URL-decoded to handle special characters,
        and then it is URL-encoded to ensure it is a valid filename.
    """
    # Extract filenames
    name = urlparse(url).path.rsplit('/', 1)[1]

    if name == None or name == '':
        return ''

    # Decode and re-encode
    return quote(unquote(name))


def init_dir(path: str, new: bool = True) -> None:
    """
    Initialize a directory, optionally creating a new one and deleting existing files.

    Args:
        path (str): The path of the directory to initialize.
        new (bool, optional): Whether to create a new directory (if it doesn't exist) and delete existing files. Default is True.

    Returns:
        None

    Note:
        This function initializes a directory by either creating a new one or leaving it as-is.
        If `new` is set to True, it will create a new directory and delete any existing files within it.
        If `new` is set to False, it will leave the directory as-is.
    """
    print()
    print('# 📦 Initialize directory')
    print('# path = ', path)
    print('# new = ', new)

    # If the directory does not exist, create it.
    os.makedirs(path, exist_ok=True)

    # Get a list of files
    name = '**'
    files = glob.glob(os.path.join(path, name), recursive=True)

    print()
    print('# 🗑️ Delete target = ', files)

    # Delete files
    shutil.rmtree(path)

    # Create New
    if new:
        os.makedirs(path, exist_ok=True)


def download_dir_locally(root_path: str, out_path: str, init: bool = True) -> None:
    """
    Compress a directory and download it to a local destination.

    Args:
        root_path (str): The path of the directory to be compressed.
        out_path (str): The path where the compressed file will be saved.
        init (bool, optional): Whether to initialize the output destination directory. Default is True.

    Returns:
        None

    Note:
        This function compresses the specified directory and downloads it to the specified destination.
        It can optionally initialize the output directory, and it uses the current timestamp to generate a unique ZIP file name.
        The compressed file is created in ZIP format.
        After compression, it initiates the download of the compressed file.

    Warning:
        This function relies on external libraries and utilities such as 'shutil.make_archive' and 'files.download'.
        Ensure that these dependencies are properly configured and available.
    """
    print()
    print('# 🪂 Compress the directory and download it.')
    print('# root_path = ', root_path)
    print('# out_path = ', out_path)

    # Get a list of files
    name = '**'
    file_list = glob.glob(os.path.join(root_path, name), recursive=True)
    print('# files = ', file_list)

    # Initialize output destination directory
    if init:
        init_dir(out_path)

    # ZIP file name
    zip_name = datetime.now(JST).strftime('_%Y%m%d_%H%M%S_archive_')

    # Compress an entire dir
    archive = shutil.make_archive(os.path.join(
        out_path, zip_name), format='zip', root_dir=root_path)
    files.download(archive)

    print()
    print('# 🙏 Please wait while downloading completes.')
    print('# 📚 Download Target = ',  archive)


def display_grid_images(path: str, name: str, width: int = 15, height: int = 15) -> list:
    """
    Display images from the specified directory in a grid.

    This function searches for image files with the specified name pattern in the given directory
    and displays them in a grid layout. It uses the Matplotlib library to create the grid of images.

    Args:
        path (str): The directory path where the image files are located.
        name (str): The name pattern of the image files to display.
        width (int, optional): The width of the entire grid display in inches. Default is 15.
        height (int, optional): The height of the entire grid display in inches. Default is 15.

    Returns:
        list: A list of file paths to the displayed images.

    """
    # Define number of image columns to display
    DISPLAY_IMAGE_COLUMNS: Final = 3

    print()
    print('# path = ', path)
    print('# name = ', name)
    print('# width = ', width)
    print('# height = ', height)
    print()

    # Get file list
    file_list = glob.glob(os.path.join(path, name), recursive=True)
    file_list.sort()

    print('# 🗃️ file list = ', file_list)
    print('# file count = ', len(file_list))
    print()

    # Number of columns
    cols = DISPLAY_IMAGE_COLUMNS

    # Number of rows
    counts = len(file_list)
    rows = (counts // cols) if counts % cols == 0 else (counts // cols + 1)

    fig = plt.figure(figsize=(width, height))
    grid = ImageGrid(fig, 111, nrows_ncols=(rows, cols), axes_pad=0)

    # Set image
    for ax, file in zip(grid, file_list):
        ax.imshow(Image.open(file))

    # Hide axes
    for ax in grid.axes_all:
        ax.axis('off')

    # Show images
    plt.show()

    return file_list


# Verify that the copy was successful
def is_dir_content_equal(dir1, dir2):
    """
    Compare the contents of two directories to check if they are equal.

    Args:
        dir1 (str): The path to the first directory.
        dir2 (str): The path to the second directory.

    Returns:
        bool: True if the directory contents are equal, False otherwise.
    """

    dirs_cmp = filecmp.dircmp(dir1, dir2)
    if dirs_cmp.left_only or dirs_cmp.right_only or dirs_cmp.diff_files:
        return False
    for common_dir in dirs_cmp.common_dirs:
        new_dir1 = os.path.join(dir1, common_dir)
        new_dir2 = os.path.join(dir2, common_dir)
        if not is_dir_content_equal(new_dir1, new_dir2):
            return False
    return True


def copy_directory(source_dir: str, destination_dir: str)-> bool:
    """
    Copy the contents of a source directory to a destination directory.

    Args:
        source_dir (str): The path to the source directory.
        destination_dir (str): The path to the destination directory.

    Returns:
        bool: True if the directory copy was successful, False otherwise.
    """

    # Source directory
    file_list = glob.glob(os.path.join(source_dir, "*.*"), recursive=True)
    file_list.sort()

    print()
    print('# source dir = ', source_dir)
    print('# 🗃️ file list = ', file_list)

    # Destination directory
    file_list = glob.glob(os.path.join(destination_dir, "*.*"), recursive=True)
    file_list.sort()

    print()
    print('# destination dir = ', destination_dir)
    print('# 🗃️ file list = ', file_list)

    # Delete destination folder if it exists
    if os.path.exists(destination_dir):
        shutil.rmtree(destination_dir)

    # Copy entire folder
    shutil.copytree(source_dir, destination_dir)

    # Verify that the copy was successful
    if is_dir_content_equal(source_dir, destination_dir):
        print()
        print('# 🆗 Directory copy completed. 💯')
        return True

    return False


class APIClient(object):
    """
    SORACOM API Client Class

    This class combines various API-related implementations into a single client class.
    It provides methods for authentication, accessing camera-related endpoints, and more.

    Args:
        endpoint_url (str): The base URL of the SORACOM API.

    Attributes:
        endpoint_url (str): The base URL of the SORACOM API.
        api_key (str): The API key used for authentication.
        token (str): The API token used for authentication.

    Note:
        This class requires external functions such as 'send_post_request' and 'send_get_request' for making API requests.
        Ensure that these functions are properly defined and available.

    Warning:
        This class is designed for interacting with the SORACOM API and relies on external libraries and utilities.
        Ensure that the necessary dependencies are correctly configured.
    """

    def __init_vars(self):
        """
        Initialize instance variables for API client.
        """
        self.endpoint_url = ''
        self.api_key = ''
        self.token = ''

    def __init__(self, endpoint_url: str):
        """
        Initialize the API client with the SORACOM API endpoint URL.

        Args:
            endpoint_url (str): The base URL of the SORACOM API.
        """
        self.__init_vars()
        self.endpoint_url = endpoint_url

    def __del__(self):
        """
        Destructor to reset instance variables.
        """
        self.__init_vars()

    def __get_auth_headers(self):
        """
        Get the authentication headers for API requests.

        Returns:
            dict: A dictionary containing the authentication headers.
        """
        return {
            'Content-Type': 'application/json',
            'X-Soracom-API-Key': self.api_key,
            'X-Soracom-Token': self.token,
        }

    def is_authenticated(self) -> bool:
        """
        Check if the API client is authenticated.

        Returns:
            bool: True if the client is authenticated with an API key and token, otherwise False.
        """
        status = False
        if self.api_key != '' and self.token != '':
            status = True
        return status

    # /auth
    # Authenticate API access and issue API key and API token
    # Reference: https://users.soracom.io/ja-jp/tools/api/reference/#/Auth/auth
    def authenticate_user(self, email: str, password: str, opid: str, name: str, mfa_code: str = None, timeout: int = 3600) -> APIResult:
        """
        Authenticate the user for API access and obtain an API key and API token.

        Args:
            email (str): The email address of the user.
            password (str): The password associated with the email address.
            opid (str): The operator ID (SAM) of the user.
            name (str): The user's name (SAM).
            mfa_code (str, optional): The Multi-Factor Authentication (MFA) code if MFA is enabled (SAM).
            timeout (int, optional): The API key expiration time in seconds (Default is 3600 seconds).

        Returns:
            APIResult: An APIResult object containing the API response.

        Note:
            This method performs user authentication and issues API key and API token.
            The authentication process can use either Root or SAM credentials based on the provided parameters.
            If both Root and SAM credentials are valid, SAM credentials are preferred.

        Reference:
            https://users.soracom.io/ja-jp/tools/api/reference/#/Auth/auth

        Warning:
            This method requires external functions such as 'send_post_request' for making API requests.
            Ensure that the necessary dependencies are correctly configured.
        """
        api_path = '/auth/'
        url = merge_url(self.endpoint_url, api_path)
        headers = {'Content-Type': 'application/json'}

        # authentication uses the following values
        # Root: email, password
        # SAM: opid, name, password
        data = dict()

        # prepare the value for the Root case
        if email is not None and email != '':
            if password is not None and password != '':
                data['email'] = email
                data['password'] = password

        # prepare the value for the SAM case
        # If both are valid, SAM is preferred
        if opid is not None and opid != '':
            if name is not None and name != '':
                if password is not None and password != '':
                    data['operatorId'] = opid
                    data['userName'] = name
                    data['password'] = password

        # set the API key expiration time (Default 1 hour)
        data['tokenTimeoutSeconds'] = timeout

        # MFA Authentication Code
        if mfa_code is not None and mfa_code != '':
            data['mfaOTPCode'] = mfa_code

        # Send Request
        return send_post_request(url, headers=headers, data=data)

    # /auth/logout
    # Disable the API key and API token for accessing the SORACOM API
    # Reference: https://users.soracom.io/ja-jp/tools/api/reference/#/Auth/logout
    def logout_user(self) -> APIResult:
        """
        Disable the API key and API token to log out from SORACOM API.

        Returns:
            APIResult: An APIResult object containing the API response.

        Reference:
            https://users.soracom.io/ja-jp/tools/api/reference/#/Auth/logout

        Warning:
            This method requires external functions such as 'send_post_request' for making API requests.
            Ensure that the necessary dependencies are correctly configured.
        """
        api_path = '/auth/logout'
        url = merge_url(self.endpoint_url, api_path)
        headers = self.__get_auth_headers()
        data = dict()

        # Send Request
        return send_post_request(url, headers=headers, data=data)

    # /sora_cam/devices
    # Get the list of sora-cam compatible cameras
    # Reference: https://users.soracom.io/ja-jp/tools/api/reference/#/SoraCam/listSoraCamDevices
    def list_sora_cam_devices(self) -> APIResult:
        """
        Get the list of sora-cam compatible cameras.

        Returns:
            APIResult: An APIResult object containing the API response.

        Reference:
            https://users.soracom.io/ja-jp/tools/api/reference/#/SoraCam/listSoraCamDevices

        Warning:
            This method requires external functions such as 'send_get_request' for making API requests.
            Ensure that the necessary dependencies are correctly configured.
        """

        api_path = '/sora_cam/devices'
        url = merge_url(self.endpoint_url, api_path)
        headers = self.__get_auth_headers()

        # Send Request
        return send_get_request(url, headers=headers)

    # /sora_cam/devices/{device_id}
    # Get information about cameras compatible with sora-cam
    # Reference: https://users.soracom.io/ja-jp/tools/api/reference/#/SoraCam/getSoraCamDevice
    def get_sora_cam_device(self, id: str) -> APIResult:
        """
        Get information about cameras compatible with sora-cam.

        Args:
            id (str): The ID of the sora-cam compatible camera.

        Returns:
            APIResult: An APIResult object containing the API response.

        Reference:
            https://users.soracom.io/ja-jp/tools/api/reference/#/SoraCam/getSoraCamDevice

        Warning:
            This method requires external functions such as 'send_get_request' for making API requests.
            Ensure that the necessary dependencies are correctly configured.
        """

        api_path = '/sora_cam/devices/' + id
        url = merge_url(self.endpoint_url, api_path)
        headers = self.__get_auth_headers()

        # Send Request
        return send_get_request(url, headers=headers)

    # /sora_cam/devices/{device_id}/exports/usage
    # Get the number of still images that can be exported from a sora-cam compatible camera and the exportable time of the recorded video
    # Reference: https://users.soracom.io/ja-jp/tools/api/reference/#/SoraCam/getSoraCamDeviceExportUsage
    def get_sora_cam_export_usage(self, id: str) -> APIResult:
        """
        Get the number of still images that can be exported from a sora-cam compatible camera and the exportable time of the recorded video.

        Args:
            id (str): The ID of the sora-cam compatible camera.

        Returns:
            APIResult: An APIResult object containing the API response.

        Reference:
            https://users.soracom.io/ja-jp/tools/api/reference/#/SoraCam/getSoraCamDeviceExportUsage

        Warning:
            This method requires external functions such as 'send_get_request' for making API requests.
            Ensure that the necessary dependencies are correctly configured.
        """

        api_path = '/sora_cam/devices/' + id + '/exports/usage'
        url = merge_url(self.endpoint_url, api_path)
        headers = self.__get_auth_headers()

        # Send Request
        return send_get_request(url, headers=headers)

    # /sora_cam/devices/{device_id}/stream
    # Get information about downloading streaming video (real-time video / recorded video)
    # Reference: https://users.soracom.io/ja-jp/tools/api/reference/#/SoraCam/getSoraCamDeviceStreamingVideo
    def get_sora_cam_streaming_video(self, id: str, time_from: int = None, time_to: int = None) -> APIResult:
        """
        Get information about downloading streaming video (real-time video / recorded video) from a sora-cam compatible camera.

        Args:
            id (str): The ID of the sora-cam compatible camera.
            time_from (int, optional): The starting time for the video stream (Unix timestamp).
            time_to (int, optional): The ending time for the video stream (Unix timestamp).

        Returns:
            APIResult: An APIResult object containing the API response.

        Note:
            This method retrieves information about downloading streaming video from a sora-cam compatible camera.
            You can specify a time range to retrieve recorded video.

        Reference: https://users.soracom.io/ja-jp/tools/api/reference/#/SoraCam/getSoraCamDeviceStreamingVideo
        """
        api_path = '/sora_cam/devices/' + id + '/stream'
        url = merge_url(self.endpoint_url, api_path)
        headers = self.__get_auth_headers()

        # Set parameters as needed.
        params = None
        if time_from is not None and time_from > 0:
            if time_to is not None and time_to > 0:
                params = {
                    'from': time_from,
                    'to': time_to,
                }

        # Send Request
        return send_get_request(url, headers=headers, params=params)

    # /sora_cam/devices/{device_id}/images/exports
    # Start the process to export still images from recorded videos stored in the cloud's continuous recording.
    # Reference: https://users.soracom.io/ja-jp/tools/api/reference/#/SoraCam/exportSoraCamDeviceRecordedImage
    def export_sora_cam_recorded_image(self, id: str, time_from: int, filter: bool = False) -> APIResult:
        """
        Start the process to export still images from recorded videos stored in the cloud's continuous recording for a sora-cam compatible camera.

        Args:
            id (str): The ID of the sora-cam compatible camera.
            time_from (int): The starting time for exporting still images (Unix timestamp).
            filter (bool, optional): Whether to apply a filter for wide-angle correction.

        Returns:
            APIResult: An APIResult object containing the API response.

        Note:
            This method initiates the process of exporting still images from recorded videos stored in the cloud's continuous recording for a sora-cam compatible camera.
            You can specify a starting time and optionally apply a wide-angle correction filter.

        Reference: https://users.soracom.io/ja-jp/tools/api/reference/#/SoraCam/exportSoraCamDeviceRecordedImage
        """
        api_path = '/sora_cam/devices/' + id + '/images/exports'
        url = merge_url(self.endpoint_url, api_path)
        headers = self.__get_auth_headers()

        data = {
            'time': time_from,
        }

        if filter:
            data['imageFilters'] = ['wide_angle_correction']

        # Send Request
        return send_post_request(url, headers=headers, data=data)

    # /sora_cam/devices/{device_id}/images/exports/{export_id}
    # Retrieve the current status of the process to export still images from recorded videos stored in the cloud's continuous recording.
    # Reference: https://users.soracom.io/ja-jp/tools/api/reference/#/SoraCam/getSoraCamDeviceExportedVideo
    def get_sora_cam_exported_image(self, id: str, export_id: str) -> APIResult:
        """
        Retrieve the current status of the process to export still images from recorded videos stored in the cloud's continuous recording for a sora-cam compatible camera.

        Args:
            id (str): The ID of the sora-cam compatible camera.
            export_id (str): The ID of the export process for still images.

        Returns:
            APIResult: An APIResult object containing the API response.

        Note:
            This method retrieves the current status of the process to export still images from recorded videos stored in the cloud's continuous recording for a sora-cam compatible camera.

        Reference: https://users.soracom.io/ja-jp/tools/api/reference/#/SoraCam/getSoraCamDeviceExportedVideo
        """
        api_path = '/sora_cam/devices/' + id + '/images/exports/' + export_id
        url = merge_url(self.endpoint_url, api_path)
        headers = self.__get_auth_headers()

        # Send Request
        return send_get_request(url, headers=headers)

    # /sora_cam/devices/{device_id}/events
    # Retrieve a list of events from SoraCam compatible cameras.
    # Reference: https://users.soracom.io/ja-jp/tools/api/reference/#/SoraCam/listSoraCamDeviceEventsForDevice
    def list_sora_cam_events_for_device(self, id: str, limit: int = None, time_from: int = None, time_to: int = None, last_key: str = None) -> APIResult:
        """
        Retrieve a list of events from a SoraCam compatible camera.

        Args:
            id (str): The ID of the SoraCam compatible camera.
            limit (int, optional): The maximum number of events to retrieve (Default is None).
            time_from (int, optional): The start time for filtering events (Default is None).
            time_to (int, optional): The end time for filtering events (Default is None).
            last_key (str, optional): The last evaluated key for paginating results (Default is None).

        Returns:
            APIResult: An APIResult object containing the API response.

        Note:
            This method retrieves a list of events from a SoraCam compatible camera. You can specify filtering criteria such as time range and pagination.

        Reference: https://users.soracom.io/ja-jp/tools/api/reference/#/SoraCam/listSoraCamDeviceEventsForDevice
        """
        api_path = '/sora_cam/devices/' + id + '/events'
        url = merge_url(self.endpoint_url, api_path)
        headers = self.__get_auth_headers()

        # Set parameters as needed.
        params = dict()

        if limit is not None and limit > 0:
            params['limit'] = limit

        if time_from is not None and time_from > 0:
            params['from'] = time_from

        if time_to is not None and time_to > 0:
            params['to'] = time_to

        if last_key is not None and last_key != '':
            params['last_evaluated_key'] = last_key

        if not params:
            params = None

        # Send Request
        return send_get_request(url, headers=headers, params=params)

    # /sora_cam/devices/{device_id}/atomcam/settings
    # Get SoraCam compatible camera settings
    # Reference: https://users.soracom.io/ja-jp/tools/api/reference/#/SoraCam/getSoraCamDeviceAtomCamSettings
    def get_sora_cam_atom_cam_settings(self, id: str) -> APIResult:
        """
        Retrieve settings for a SoraCam compatible camera's AtomCam feature.

        Args:
            id (str): The ID of the SoraCam compatible camera.

        Returns:
            APIResult: An APIResult object containing the API response.

        Note:
            This method retrieves settings related to the AtomCam feature of a SoraCam compatible camera.

        Reference: https://users.soracom.io/ja-jp/tools/api/reference/#/SoraCam/getSoraCamDeviceAtomCamSettings
        """
        api_path = '/sora_cam/devices/' + id + '/atomcam/settings'
        url = merge_url(self.endpoint_url, api_path)
        headers = self.__get_auth_headers()

        # Send Request
        return send_get_request(url, headers=headers)

    # /sora_cam/devices/{device_id}/recordings_and_events
    # Get a list of time periods and a list of events recorded by SoraCam compatible camera.
    # Reference: https://users.soracom.io/ja-jp/tools/api/reference/#/SoraCam/listSoraCamDeviceRecordingsAndEvents
    def list_sora_cam_recordings_and_events(self, id: str, time_from: int = None, time_to: int = None, last_key: str = None) -> APIResult:
        """
        Retrieve a list of time periods and events recorded by a SoraCam compatible camera.

        Args:
            id (str): The ID of the SoraCam compatible camera.
            time_from (int, optional): The starting timestamp for filtering events (Unix timestamp in seconds).
            time_to (int, optional): The ending timestamp for filtering events (Unix timestamp in seconds).
            last_key (str, optional): The last evaluated key for paginating results.

        Returns:
            APIResult: An APIResult object containing the API response.

        Note:
            This method retrieves a list of time periods and events recorded by a SoraCam compatible camera.
            You can specify time range filters and use pagination with the 'last_key' parameter.

        Reference: https://users.soracom.io/ja-jp/tools/api/reference/#/SoraCam/listSoraCamDeviceRecordingsAndEvents
        """
        api_path = '/sora_cam/devices/' + id + '/recordings_and_events'
        url = merge_url(self.endpoint_url, api_path)
        headers = self.__get_auth_headers()

        # Set parameters as needed.
        params = dict()

        if time_from is not None and time_from > 0:
            params['from'] = time_from

        if time_to is not None and time_to > 0:
            params['to'] = time_to

        if last_key is not None and last_key != '':
            params['last_evaluated_key'] = last_key

        if len(params) <= 0:
            params = None

        # Send Request
        return send_get_request(url, headers=headers, params=params)


#
# --------------- Execution ---------------
#


# Process start message display
print_start_msg()

print()
print('# ℹ️ Python Version = ', sys.version)

# Process end message display
print_end_msg()


In [ ]:
# @title ステップ 3: OpenAI API の API キーを入力する

#
# --------------- Definition ---------------
#

# @markdown # Fill in the required information and run.
# @markdown - - -

# @markdown Login using OpenAI Organization ID.
use_organization = False  # @param {type:"boolean"}

# @markdown - - -

import ipywidgets as widgets
from openai import OpenAI

# OpenAI Client
_openai_client: OpenAI = None


# Retrieve the OpenAI instance with API key configured.
def get_openai_client() -> OpenAI:
    """
    Get the OpenAI client instance with the configured API key.

    Returns:
        OpenAI: An instance of the OpenAI client.
    """
    return _openai_client

# Set the API key for the OpenAI instance.
def set_openai_client(api_key: str, organization: str):
    """
    Set the API key and organization for the OpenAI client instance.

    Args:
        api_key (str): The API key for authenticating with OpenAI.
        organization (str): The organization associated with the API key.
    """
    global _openai_client
    _openai_client = OpenAI(api_key=api_key, organization=organization)

# Delete an instance of OpenAI
def delete_openai_client():
    global _openai_client
    if _openai_client is not None:
        del _openai_client


# Check if the API key is valid.
def check_openai_client() -> bool:
    """
    Check if the OpenAI API client is configured correctly and the API key is valid.

    Returns:
        bool: True if the API key is valid, False otherwise.
    """
    # Verify if authenticated correctly
    if _openai_client.organization is not None and _openai_client.organization != '':
        print()
        print('# 🆔 OpenAI Organization ID = ', _openai_client.organization)

    if len(_openai_client.models.list().data) <= 0:
        print()
        print('➕➕➕ ⛔ Could not confirm operation with the entered API key. Please check your input. ➕➕➕')
        return False

    print()
    print('# 📜 Number of all AI models = ', len(_openai_client.models.list().data))
    print()
    print('# 🔑 API access has been verified. 💯')

    return True


def show_openai_api_widgets(use_organization: bool):
    """
    Display OpenAI API authentication widgets.

    Args:
        use_organization (bool): True if an Organization ID is required, False otherwise.

    Returns:
        None
    """
    # Prepare UI for OpenAI API Keys
    # Organization ID input field
    input_openai_org = widgets.Text(
        value=None,
        placeholder='Enter Organization ID',
        disabled=False
    )

    # API Key input field
    input_opneai_api_key = widgets.Password(
        value=None,
        placeholder='Enter API Key',
        disabled=False
    )

    # Authentication button
    button_auth_openai = widgets.Button(
        description='Apply',
        disabled=False,
        button_style='info',  # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Apply',
        icon='rocket'  # (FontAwesome names without the `fa-` prefix)
    )

    # UI display
    print()
    print('➕➕➕ 🔰 Use the OpenAI API. Fill in the required fields. Then press the Apply button. ➕➕➕')
    print()

    if use_organization:
        display(widgets.Label(value="🆔 Organization ID: "))
        display(input_openai_org)

    display(widgets.Label(value="🔑 API Key: "))
    display(input_opneai_api_key)

    print()
    display(button_auth_openai)


    def on_button_auth_openai_handler(change):
        """
        Handle authentication when the Apply button is pressed.

        This function is called when the Apply button for OpenAI authentication is pressed.
        It reads the values of the Organization ID and API Key input fields, validates them,
        and then authenticates with the OpenAI API using the provided credentials.

        Args:
            change: The change event when the Apply button is pressed.

        """
        api_key = input_opneai_api_key.value
        org_Id = input_openai_org.value

        # Check input values
        if api_key is None or api_key == '':
            print()
            print('➕➕➕ ⛔ OpenAI API key has not been entered. Please check your input. ➕➕➕')
            return

        if use_organization:
            if org_Id is None or org_Id == '':
                print()
                print('➕➕➕ ⛔ OpenAI Organization ID has not been entered. Please check your input. ➕➕➕')
                return

        # Set authentication keys OpenAI client
        set_openai_client(api_key, org_Id)

        # Verify if authenticated correctly
        if not check_openai_client():
            return

        # Clear input values to prevent reuse
        org_Id = ''
        input_openai_org.value = ''
        api_key = ''
        input_opneai_api_key.value = ''

        # Process end message display
        print_end_msg()

    # Handler Setup
    button_auth_openai.on_click(on_button_auth_openai_handler)


#
# --------------- Execution ---------------
#


# Process start message display
print_start_msg()

# Show the OpenAI API key input UI
show_openai_api_widgets(use_organization)

# Process end message display
print_end_msg()


In [ ]:
# @title ステップ 4: 利用する AI モデルを選択する

#
# --------------- Definition ---------------
#

# The selected AI model
_selected_openai_model: str = None


# Get the selected AI model name.
def get_selected_openai_model() -> str:
    """
    Get the name of the selected AI model.

    Returns:
        str: The name of the selected AI model.
    """
    return _selected_openai_model

# Set the selected AI model.
def set_selected_openai_model(model: str):
    """
    Set the selected AI model.

    Args:
        model (str): The name of the AI model to be selected.
    """
    global _selected_openai_model
    _selected_openai_model = model

    print()
    print('# ✅ Selected AI model name = ', _selected_openai_model)


def get_openai_vision_models(openai_client: OpenAI) -> list:
    """
    Get a list of vision-capable AI models available in OpenAI.

    This function retrieves a list of AI models from the OpenAI API, filters them to include only
    vision-capable models (models with names starting with 'gpt-' and containing '-vision'),
    and returns the sorted list of vision models.

    Args:
        openai_client (OpenAI): An authenticated OpenAI client.

    Returns:
        list: A list of vision-capable AI model names.

    """
    # Get a list of models
    print()
    print('# 🆕 All AI models = ', openai_client.models.list().data)

    # Display only vision-capable models.
    ai_model_list = list()
    for item in openai_client.models.list().data:
        if item.id.startswith('gpt-'):
            if item.id.find('-vision') != -1:
                ai_model_list.append(item.id)
    ai_model_list.sort(reverse=True)

    print('# 🏞️ Vision models: ', ai_model_list)

    return ai_model_list


def show_openai_models_widgets(ai_model_list: list):
    """
    Display a widget to select from a list of vision models.

    This function creates a Dropdown widget displaying a list of vision models and allows the user
    to select a model from the list.

    Args:
        ai_model_list (list): A list of vision-capable AI model names.

    """
    # Models Dropdown list
    select_ai_models = widgets.Dropdown(
        options=ai_model_list,
        disabled=False,
        value=ai_model_list[0],
    )

    print()
    print('➕➕➕ 🔰 Select from Vision models ➕➕➕')
    print()
    display(widgets.Label(value="🏞️ Vision models： "))
    display(select_ai_models)


    # Function called when a Dropdown is selected
    def on_select_ai_models_handler(change):
        """
        Handle the event when a model is selected from a Dropdown.

        Args:
            change (dict): The change event object.

        """
        selected_item = change['new']

        # Maintain information about selected models
        set_selected_openai_model(selected_item)

        # Process end message display
        print_end_msg()

    # Handler Setup
    select_ai_models.observe(on_select_ai_models_handler, names='value')


#
# --------------- Execution ---------------
#


# Process start message display
print_start_msg()

# get OpenAI client
openai_client = get_openai_client()

# Retrieve the list of vision models provided by OpenAI.
openai_vision_models = get_openai_vision_models(openai_client)

# Display the UI to select an AI model.
show_openai_models_widgets(openai_vision_models)

# Select the first model in the list
if len(openai_vision_models) == 1:
    set_selected_openai_model(openai_vision_models[0])

# Process end message display
print_end_msg()


In [ ]:
# @title ステップ 5: SORACOM API にログインする

#
# --------------- Definition ---------------
#

# @markdown # Fill in the required information and run.
# @markdown - - -

# @markdown Set the SORACOM API endpoint.
endpoint_url = 'https://api.soracom.io/v1'  # @param {type:"string"}

# @markdown Select user to login to SORACOM User Console.
login_type = "SAM User"  # @param ["Root User", "SAM User"]

# @markdown Login using multi-factor authentication.
use_mfa = False  # @param {type:"boolean"}

# @markdown - - -


# init SORACOM API client
# We will continue to use this variable when calling the API in the following sections
_soracom_api_client: APIClient = APIClient(endpoint_url)


# Get the SORACOM API client.
def get_soracom_api_client() -> APIClient:
    """
    Retrieve the SORACOM API client instance.

    Returns:
        APIClient: The SORACOM API client instance.

    """
    return _soracom_api_client

# Set the SORACOM API client.
def set_soracom_api_client(apiKey: str, token: str):
    """
    Set the SORACOM API client with the provided API key and token.

    Args:
        apiKey (str): The API key for authentication.
        token (str): The token for authentication.

    Returns:
        None
    """
    global _soracom_api_client
    # Keep your API keys
    _soracom_api_client.api_key = apiKey
    _soracom_api_client.token = token

# Logout and Delete of SORACOM API client
def logout_delete_soracom_api_client():
    """
    Logout from the SORACOM API and delete the SORACOM API client instance if it's authenticated.

    This function checks if the SORACOM API client is authenticated and attempts to logout from the API.
    If the logout is successful, it deletes the SORACOM API client instance. If the logout fails, an error
    message is displayed.

    Returns:
        None
    """
    global _soracom_api_client
    if _soracom_api_client is not None:
        if _soracom_api_client.is_authenticated():
            api_result = _soracom_api_client.logout_user()

            if api_result.check_success():
                print()
                print('# 👋 SORACOM API logout is completed. 💯')
                del _soracom_api_client
            else:
                print()
                print('➕➕➕ ⛔ SORACOM API logout failed. Please check the error message shown on the display. ➕➕➕')


def show_soracom_login_widgets(soracom_api_client: APIClient, login_type: str, use_mfa: bool):
    """
    Display the SORACOM login widgets based on login type and MFA usage.

    Args:
        login_type (str): The login type, either 'Root User' or 'SAM User'.
        use_mfa (bool): True if MFA (Multi-Factor Authentication) is enabled, False otherwise.
        soracom_api_client (APIClient): The SORACOM API client for making authentication requests.

    Returns:
        None
    """

    # Prepare UI for login
    # Root User email address input field
    input_email = widgets.Text(
        value=None,
        placeholder='Enter Email Address',
        disabled=False
    )

    # SAM User operator ID input field
    input_operator_Id = widgets.Text(
        value=None,
        placeholder='Enter Operator ID',
        disabled=False
    )

    # SAM User user name input field
    input_user_name = widgets.Text(
        value=None,
        placeholder='Enter User Name',
        disabled=False
    )

    # User password input field
    input_password = widgets.Password(
        value=None,
        placeholder='Enter Password',
        disabled=False
    )

    # MFA code input field
    input_mfa_code = widgets.Text(
        value=None,
        placeholder='Enter MFA Code',
        disabled=False
    )

    # Token timeout input slider
    slider_token_timeout = widgets.IntSlider(
        value=3600,
        min=1,
        max=172800,
        step=1,
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True,
        readout_format='d',
    )

    # Login button
    login_button = widgets.Button(
        description='Login',
        disabled=False,
        button_style='info',  # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Login',
        icon='user'  # (FontAwesome names without the `fa-` prefix)
    )

    # Logout button
    logout_button = widgets.Button(
        description='Logout',
        disabled=False,
        button_style='warning',  # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Logout',
        # (FontAwesome names without the `fa-` prefix)
        icon='right-from-bracket'
    )

    # UI display by login type
    if login_type == 'Root User':
        print()
        print('➕➕➕ 🔰 Login as the Root User. Fill in the required fields. Then press the Login button. ➕➕➕')
        print()

        display(widgets.Label(value="📧 Email Address: "))
        display(input_email)

    elif login_type == 'SAM User':
        print()
        print('➕➕➕ 🔰 Login as the SAM User. Fill in the required fields. Then press the Login button. ➕➕➕')
        print()

        display(widgets.Label(value="🪪 Operator ID: "))
        display(input_operator_Id)
        display(widgets.Label(value="📛 User Name: "))
        display(input_user_name)

    display(widgets.Label(value="🔑 Password: "))
    display(input_password)

    if use_mfa:
        display(widgets.Label(value="🔒 MFA Code: "))
        display(input_mfa_code)

    display(widgets.Label(value="⌛ API Token Timeout Seconds : "))
    display(slider_token_timeout)

    print()
    display(login_button)


    # Function called when a button is pressed
    def on_login_button_handler(change):
        """
        Handle the login button click event.

        This function performs user authentication with the SORACOM API based on the provided login information.
        It checks the input values, performs authentication, and handles the API response.
        If authentication is successful, it sets the API key and token for future API requests.

        Args:
            change (dict): A dictionary containing information about the button click event.

        Returns:
            None
        """

        operator_Id = input_operator_Id.value
        user_name = input_user_name.value
        password = input_password.value
        email = input_email.value
        mfa_code = None
        token_timeout = slider_token_timeout.value

        # Check input values
        if login_type == 'Root User':
            if email == None or email == '' or password == None or password == '':
                print()
                print('➕➕➕ ⛔ The login information for the Root User is missing. Please check your input. ➕➕➕')
                return
        elif login_type == 'SAM User':
            if operator_Id == None or operator_Id == '' or password == None or password == '' or user_name == None or user_name == '':
                print()
                print('➕➕➕ ⛔ The login information for the SAM User is missing. Please check your input. ➕➕➕')
                return

        # Check MFA Authentication Code
        if use_mfa:
            mfa_code = input_mfa_code.value
            if not (mfa_code.isdecimal() and mfa_code.isascii()):
                print()
                print('➕➕➕ ⛔ Please enter only numbers for the MFA verification code. ➕➕➕')
                return

        # Perform Authentication
        api_result = soracom_api_client.authenticate_user(
            email, password, operator_Id, user_name, mfa_code, token_timeout)

        if not api_result.check_success():
            print()
            print('➕➕➕ ⛔ API authentication failed. Please check your input. ➕➕➕')
            return

        auth_resp = api_result.response
        if auth_resp['apiKey'] is not None and auth_resp['apiKey'] != '':
            if auth_resp['token'] is not None and auth_resp['token'] != '':
                # Set your API keys
                set_soracom_api_client(auth_resp['apiKey'], auth_resp['token'])

                print()
                print('# 🔑 API access has been authenticated 💯')
                print()
                print('# ⏳ API Token Timeout Seconds = ', token_timeout,
                      datetime.now(JST) + timedelta(seconds=token_timeout))

                # Clear input values to prevent reuse
                operator_Id = ''
                input_operator_Id.value = ''
                user_name = ''
                input_user_name.value = ''
                password = ''
                input_password.value = ''
                email = ''
                input_email.value = ''
                mfa_code = ''
                input_mfa_code.value = ''
                token_timeout = 0
                slider_token_timeout.value = 3600

        # Process end message display
        print_end_msg()

    # Handler Setup
    login_button.on_click(on_login_button_handler)


#
# --------------- Execution ---------------
#


# Process start message display
print_start_msg()

# Get the SORACOM API client.
soracom_api_client = get_soracom_api_client()

# Show the login UI for the SORACOM API.
show_soracom_login_widgets(soracom_api_client, login_type, use_mfa)

# Process end message display
print_end_msg()


In [ ]:
# @title ステップ 6: カメラを 1 台選択する

#
# --------------- Definition ---------------
#

# @markdown # Fill in the required information and run.
# @markdown - - -

# @markdown Display only connectable cameras.
connectable_cameras = True  # @param {type:"boolean"}

# @markdown - - -

# The selected Soracom compatible camera.
_selected_camera: dict = None


def get_selected_camera() -> dict:
    """
    Get the selected camera information.

    Returns:
        dict: A dictionary representing the selected camera.
    """
    return _selected_camera

def get_selected_camera_id() -> str | None:
    """
    Get the ID of the selected camera.

    Returns:
        str | None: The ID of the selected camera or None if no camera is selected.
    """
    return _selected_camera.get('deviceId')

def set_selected_camera(camera: dict):
    """
    Set the selected camera.

    Args:
        camera (dict): A dictionary representing the selected camera.
    """
    global _selected_camera
    _selected_camera = camera

    print()
    print('# ✅ Selected camera ID = ', _selected_camera.get('deviceId'))
    print('# 📸 Selected camera name = ', _selected_camera.get('name'))

def get_camera_list(soracom_api_client: APIClient) -> list | None:
    """
    Get a list of cameras using the SORACOM API client.

    Args:
        soracom_api_client (APIClient): The SORACOM API client.

    Returns:
        list | None: A list of cameras retrieved from the SORACOM API, or None if the request failed.
    """
    api_result = soracom_api_client.list_sora_cam_devices()

    if api_result.check_success():
        print()
        print('# 📷 Camera List = ', api_result.response)
        print('# 📜 Number of All Camera List = ', len(api_result.response))

        return api_result.response

def get_camera_details(soracom_api_client: APIClient, camera_id: str) -> dict | None:
    """
    Get detailed information about a camera device.

    This function retrieves detailed information about a camera device based on its unique `camera_id`. It makes an API
    request to the SORACOM API using the provided `soracom_api_client` and returns the response if the request is
    successful.

    Args:
        camera_id (str): The unique identifier of the camera device.
        soracom_api_client (APIClient): The SORACOM API client used to make the API request.

    Returns:
        dict or None: A dictionary containing detailed information about the camera device if the request is successful.
            Returns None if the request fails.

    """
    api_result = soracom_api_client.get_sora_cam_device(camera_id)

    if api_result.check_success():
        print()
        print('# 📸 Camera Details = ', api_result.get_display_response())
        return api_result.response

def get_atom_cam_settings(soracom_api_client: APIClient, camera_id: str) -> dict | None:
    """
    Get ATOM Cam settings for a camera device.

    This function retrieves ATOM Cam settings for a camera device based on its unique `camera_id`. It makes an API request
    to the SORACOM API using the provided `soracom_api_client` and returns the response if the request is successful.

    Args:
        camera_id (str): The unique identifier of the camera device.
        soracom_api_client (APIClient): The SORACOM API client used to make the API request.

    Returns:
        dict or None: A dictionary containing ATOM Cam settings for the camera device if the request is successful.
            Returns None if the request fails.

    """
    api_result = soracom_api_client.get_sora_cam_atom_cam_settings(camera_id)

    if api_result.check_success():
        print()
        print('# 📷 ATOM Cam Settings = ', api_result.get_display_response())
        return api_result.response

# Time remaining for video export
def get_remaining_export_time(soracom_api_client: APIClient, camera_id: str) -> dict | None:
    """
    Get the remaining time for video export of a camera device.

    This function retrieves the remaining time for video export of a camera device with the specified `camera_id`.
    It makes an API request to the SORACOM API using the provided `soracom_api_client` and returns the response if the
    request is successful.

    Args:
        camera_id (str): The unique identifier of the camera device.
        soracom_api_client (APIClient): The SORACOM API client used to make the API request.

    Returns:
        dict or None: A dictionary containing information about the remaining time for video export if the request is
            successful. The dictionary includes details such as the remaining seconds, hours, minutes, and seconds.
            Returns None if the request fails.

    """
    api_result = soracom_api_client.get_sora_cam_export_usage(camera_id)

    if api_result.check_success():
        print('')
        print('# 🆓 Exportable Times = ', api_result.get_display_response())
        print('# 🔎 Time remaining of month = ', '{} hours {} minutes {} seconds'.format(
            int(api_result.response['video']['remainingSeconds'] / 3600),
            int((api_result.response['video']['remainingSeconds'] % 3600) / 60),
            int(api_result.response['video']['remainingSeconds'] % 60)))
        return api_result.response


def show_cameras_widgets(soracom_api_client: APIClient, camera_list: list, connectable_cameras: bool):
    """
    Display a widget to select cameras from a list.

    This function generates a dropdown list of cameras, allowing users to select a camera. When a camera is selected,
    it triggers a series of actions including getting camera details, retrieving ATOM Cam settings, displaying
    remaining video export time, and maintaining information about the selected camera.

    Args:
        camera_list (list): A list of cameras to display in the dropdown.
        connectable_cameras (bool): A flag indicating whether to display only connectable cameras.
        soracom_api_client (APIClient): An instance of the SORACOM API client.

    Returns:
        None
    """

    def create_device_list_for_dropdown(camera_list: list, connectable_cameras: bool) -> list:
        """
        Create a list of camera devices for the dropdown widget.

        Args:
            camera_list (list): A list of camera devices.
            connectable_cameras (bool): A flag indicating whether to include only connectable cameras.

        Returns:
            list: A list of tuples containing camera information for the dropdown options.
        """
        result = list()

        if len(camera_list) <= 0:
            return result

        for item in camera_list:
            if connectable_cameras:
                if not item['connected']:
                    continue

            # The following order according to the user console display
            # name / connected / firmwareVersion / productDisplayName / deviceId
            status = '🌈 Connected' if item['connected'] else '🌀 Disconnected'
            display = '{} / {} / {} / {} / {}'.format(
                item['name'], status, item['firmwareVersion'], item['productDisplayName'], item['deviceId'])
            result.append((display, item))

        return result

    # Camera Dropdown list
    select_devices = widgets.Dropdown(
        options=create_device_list_for_dropdown(camera_list, connectable_cameras),
        disabled=False,
            value=None,
        )

    print()
    print('➕➕➕ 🔰 Select from Camera List ➕➕➕')
    print()
    display(widgets.Label(value="📷 Camera List: "))
    display(select_devices)


    def on_select_devices_handler(change):
        """
        Handle camera selection event.

        This function is called when a camera is selected from the dropdown list. It retrieves the selected camera's ID
        and triggers a series of actions including getting camera details, retrieving ATOM Cam settings, displaying
        remaining video export time, and maintaining information about the selected camera.

        Args:
            change (dict): A dictionary containing information about the camera selection event.

        Returns:
            None

        """

        selected_item = change['new']
        camera_id = selected_item['deviceId']

        # Get details about the selected camera
        cmera_info = get_camera_details(soracom_api_client, camera_id)

        # Get ATOM Cam Setting about the selected camera
        if cmera_info.get('connected'):
            get_atom_cam_settings(soracom_api_client, camera_id)

        # Display remaining time for video export
        get_remaining_export_time(soracom_api_client, camera_id)

        # Maintain information about selected devices
        set_selected_camera(cmera_info)

        # Process end message display
        print_end_msg()

    # Handler Setup
    select_devices.observe(on_select_devices_handler, names='value')


#
# --------------- Execution ---------------
#


# Process start message display
print_start_msg()

# Get the SORACOM API client.
soracom_api_client = get_soracom_api_client()

# Retrieve the list of cameras associated with the OPID.
camera_list = get_camera_list(soracom_api_client)

# Show the UI to select a camera.
show_cameras_widgets(soracom_api_client, camera_list, connectable_cameras)

# Select the first camera in the list
if len(camera_list) == 1:
    camera_id = camera_list[0].get('deviceId')

    # Get details about the selected camera
    cmera_info = get_camera_details(soracom_api_client, camera_id)

    # Get ATOM Cam Setting about the selected camera
    if cmera_info.get('connected'):
        get_atom_cam_settings(soracom_api_client, camera_id)

    # Display remaining time for video export
    get_remaining_export_time(soracom_api_client, camera_id)

    # Maintain information about selected devices
    set_selected_camera(cmera_info)

# Process end message display
print_end_msg()


In [ ]:
# @title ステップ 7: イベント一覧の取得範囲を指定する

#
# --------------- Definition ---------------
#

# Define variables to hold values
_selected_event_time: ExportTimeInfo = ExportTimeInfo(None, None)


def get_selected_event_time() -> ExportTimeInfo:
    """
    Get the selected event time range.

    This function returns the selected event time range, which is stored in the `_selected_event_time` variable.

    Returns:
        ExportTimeInfo: The selected event time range information.
    """
    return _selected_event_time

def set_selected_event_time(time_from: datetime, time_to: datetime):
    """
    Set the selected event time range.

    This function sets the selected event time range using the provided start and end datetime objects.
    The event time range is stored in the `_selected_event_time` variable.

    Args:
        time_from (datetime): The start datetime for the event time range.
        time_to (datetime): The end datetime for the event time range.

    Returns:
        None
    """
    global _selected_event_time
    _selected_event_time.time_from = time_from
    _selected_event_time.time_to = time_to

    print()
    if _selected_event_time.valid_vars():
        print('# 🗓️ Set the start and end times for events. 🆗')
        print()
        print('# Start date time = ', _selected_event_time.time_from)
        print('# End date time = ', _selected_event_time.time_to)
        print('# Export range = ', _selected_event_time.time_to - _selected_event_time.time_from)
    else:
        print('# 🔋 Set the retrieve all events. 🆗')


def show_event_time_range_widgets():
    """
    Display widgets for setting the time period for retrieving events.

    This function prepares and displays user interface widgets for setting the time period
    to retrieve events. It includes options to specify start and end dates and times, and
    provides buttons to apply the selected time period or retrieve events for the full term.
    """

    # Prepare a list since time picker is not available
    hours = [i for i in range(24)]
    minutes = [i for i in range(60)]
    seconds = [i for i in range(60)]
    jst_now = datetime.now(JST)
    one_hour_ago = jst_now - timedelta(hours=1)

    # Start date
    event_start_date = widgets.DatePicker(
        description='Start date:',
        value=one_hour_ago.date(),
        disabled=False
    )

    # Start time
    event_start_hours = widgets.Dropdown(
        options=hours,
        value=one_hour_ago.hour,
        description='Start time:',
        disabled=False,
    )

    event_start_minutes = widgets.Dropdown(
        options=minutes,
        value=one_hour_ago.minute,
        disabled=False,
    )

    event_start_seconds = widgets.Dropdown(
        options=seconds,
        value=one_hour_ago.second,
        disabled=False,
    )

    # End date
    event_end_date = widgets.DatePicker(
        description='End date:',
        value=jst_now.date(),
        disabled=False
    )

    # End time
    event_end_hours = widgets.Dropdown(
        options=hours,
        value=jst_now.hour,
        description='End time:',
        disabled=False,
    )

    event_end_minutes = widgets.Dropdown(
        options=minutes,
        value=jst_now.minute,
        disabled=False,
    )

    event_end_seconds = widgets.Dropdown(
        options=seconds,
        value=jst_now.second,
        disabled=False,
    )

    event_time_button = widgets.Button(
        description='Time setting',
        disabled=False,
        button_style='info',  # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Time setting',
        icon='clock'  # (FontAwesome names without the `fa-` prefix)
    )

    event_full_button = widgets.Button(
        description='Full term',
        disabled=False,
        button_style='success',  # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Full term',
        icon='gas-pump'  # (FontAwesome names without the `fa-` prefix)
    )

    # Display the time setting UI
    print()
    print('➕➕➕ 🔰  Set the period for retrieving events. ➕➕➕')
    print()
    display(widgets.HBox([event_start_date, event_start_hours, widgets.Label(
        value='：'), event_start_minutes, widgets.Label(value='：'), event_start_seconds]))
    display(widgets.HBox([event_end_date, event_end_hours, widgets.Label(
        value='：'), event_end_minutes, widgets.Label(value='：'), event_end_seconds]))
    print()
    display(widgets.HBox(
        [event_time_button, widgets.Label(value='  '), event_full_button]))

    # Function called when a button is pressed
    def on_event_time_button_handler(change):
        """
        Handle the event when the "Time setting" button is pressed.

        This function is called when the "Time setting" button is pressed by the user. It retrieves
        the selected start and end date and time, converts them to ISO format, and prints the
        selected time range and duration. It also performs checks on the entered time and updates
        the global `event_time_info` variable with the selected time period.

        Args:
            change (dict): The change dictionary containing information about the button press.

        Returns:
            None: This function does not return a value.
        """
        start_date = event_start_date.value
        start_hour = event_start_hours.value
        start_minute = event_start_minutes.value
        start_second = event_start_seconds.value

        # ISO format for start time
        # '2018-12-31T05:00:30.001000+09:00'
        start_datetime_iso = '{}T{:0=2}:{:0=2}:{:0=2}.000000+09:00'.format(
            start_date, start_hour, start_minute, start_second)
        start_datetime = datetime.fromisoformat(start_datetime_iso)

        end_date = event_end_date.value
        end_hour = event_end_hours.value
        end_minute = event_end_minutes.value
        end_second = event_end_seconds.value

        # ISO format for end time
        # '2018-12-31T05:00:30.001000+09:00'
        end_datetime_iso = '{}T{:0=2}:{:0=2}:{:0=2}.000000+09:00'.format(
            end_date, end_hour, end_minute, end_second)
        end_datetime = datetime.fromisoformat(end_datetime_iso)

        # Check the entered time
        if not ExportTimeInfo(start_datetime, end_datetime).check_time_limits(False):
            return

        # Holds the entered value
        set_selected_event_time(start_datetime, end_datetime)

        # Process end message display
        print_end_msg()


    # Function called when a button is pressed
    def on_event_full_button_handler(change):
        """
        Handle the event when the "Full term" button is pressed.

        This function is called when the "Full term" button is pressed by the user. It sets the
        `event_time_info` variable to represent retrieving all events, and it prints a message
        indicating that all events will be retrieved.

        Args:
            change (dict): The change dictionary containing information about the button press.

        Returns:
            None: This function does not return a value.
        """

        # It is treated as full time if no time is specified.
        set_selected_event_time(None, None)

        # Process end message display
        print_end_msg()

    # Handler Setup
    event_time_button.on_click(on_event_time_button_handler)

    # Handler Setup
    event_full_button.on_click(on_event_full_button_handler)


#
# --------------- Execution ---------------
#


# Process start message display
print_start_msg()

# Show the UI to specify the time period for searching events.
show_event_time_range_widgets()

# Process end message display
print_end_msg()


In [ ]:
# @title ステップ 8: 指定した範囲のイベント一覧から 1 つ選択する

#
# --------------- Definition ---------------
#

# @markdown # Fill in the required information and run.
# @markdown - - -

# @markdown Only display events that are streamable.
streamable_events = False  # @param {type:"boolean"}

# @markdown - - -

# The selected event.
_selected_event: dict = None


def get_selected_event() -> dict:
    """
    Get the selected event.

    This function returns the currently selected event.

    Returns:
        dict: The selected event as a dictionary.

    """
    return _selected_event

def set_selected_event(event: dict):
    """
    Set the selected event.

    This function sets the selected event to the provided event dictionary.

    Args:
        event (dict): The event to be set as the selected event.

    """
    global _selected_event
    _selected_event = event

    print()
    print('# ✅ Selected event = ', json.dumps(_selected_event, indent=4, ensure_ascii=False))


def get_event_list(soracom_api_client: APIClient, camera_id: str, limit: int = None, time_from: int = None, time_to: int = None, last_key: str = None) -> list:
    """
    Get a list of events for a specific camera.

    This function retrieves a list of events for the specified camera using the SORACOM API. You can specify optional
    parameters to filter the events.

    Args:
        camera_id (str): The ID of the camera for which to retrieve events.
        soracom_api_client (APIClient): The SORACOM API client.
        limit (int, optional): The maximum number of events to retrieve in a single request.
        time_from (int, optional): The start time (timestamp) to filter events. Default is None (no filter).
        time_to (int, optional): The end time (timestamp) to filter events. Default is None (no filter).
        last_key (str, optional): The last key from a previous request to fetch the next set of events. Default is None.

    Returns:
        list: A list of event objects.

    """
    result = list()

    api_result = soracom_api_client.list_sora_cam_events_for_device(
        camera_id, limit, time_from, time_to, last_key)

    if api_result.check_success():
        result.extend(api_result.response)

        # Is there more to the list?
        if api_result.additions.get('headers'):
            if api_result.additions.get('headers').get('X-Soracom-Next-Key'):
                next_key = api_result.additions.get(
                    'headers').get('X-Soracom-Next-Key')
                print()
                print('# ⏭ next_key = ', next_key)

                result.extend(get_event_list(
                    soracom_api_client, camera_id, limit, time_from, time_to, next_key))

    return result


def show_event_list_widgets(event_list: list, streamable_events: bool):
    """
    Display a widget to select from a list of events.

    This function creates a widget to select events from a given list. It filters events based on the
    `streamable_events` parameter and displays the event list as a dropdown. When an event is selected,
    it maintains information about the selected event and triggers an action to process the selected event.

    Args:
        event_list (list): A list of events to be displayed in the dropdown.
        streamable_events (bool): A flag to filter events based on their stream availability.

    Returns:
        None
    """

    def create_event_list_for_dropdown(event_list: list, streamable_events: bool) -> list:
        """
        Create a filtered list of events for a dropdown widget.

        This function takes a list of events and filters them based on the stream availability
        specified by the `streamable_events` flag. It constructs a list of display strings for
        the dropdown, including event date, type, status, image availability, and stream availability.

        Args:
            event_list (list): A list of events to be filtered and displayed.
            streamable_events (bool): A flag to filter events based on their stream availability.

        Returns:
            list: A list of tuples where each tuple contains a display string and the corresponding event item.
        """
        result = list()

        if len(event_list) <= 0:
            return result

        for item in event_list:
            # The following order according to display
            # date / type / status / image / stream
            date_time = datetime.fromtimestamp(item['time'] / 1000, JST)

            event_type = 'other'
            if item.get('eventInfo'):
                if item.get('eventInfo').get('atomEventV1'):
                    if item.get('eventInfo').get('atomEventV1').get('type'):
                        event_type = item.get('eventInfo').get(
                            'atomEventV1').get('type')

            status = ''
            if item.get('eventInfo'):
                if item.get('eventInfo').get('atomEventV1'):
                    if item.get('eventInfo').get('atomEventV1').get('recordingStatus'):
                        status = item.get('eventInfo').get(
                            'atomEventV1').get('recordingStatus')
                        if status == 'completed':
                            status = '✅ completed'

            image = 'unavailable'
            if item.get('eventInfo'):
                if item.get('eventInfo').get('atomEventV1'):
                    if item.get('eventInfo').get('atomEventV1').get('picture'):
                        image = item.get('eventInfo').get(
                            'atomEventV1').get('picture')
                        if image is not None and image != '':
                            image = '🖼️ available'

            start = 0
            if item.get('eventInfo'):
                if item.get('eventInfo').get('atomEventV1'):
                    if item.get('eventInfo').get('atomEventV1').get('startTime'):
                        start = item.get('eventInfo').get(
                            'atomEventV1').get('startTime')

            end = 0
            if item.get('eventInfo'):
                if item.get('eventInfo').get('atomEventV1'):
                    if item.get('eventInfo').get('atomEventV1').get('endTime'):
                        end = item.get('eventInfo').get(
                            'atomEventV1').get('endTime')

            stream = 'unavailable'
            stream_state = False
            if start > 0 and end > 0:
                stream = '🎥 available'
                stream_state = True

            display = '{} / {} / {} / {} / {}'.format(
                date_time, event_type, status, image, stream)
            add_state = True

            # Perform filtering?
            if streamable_events:
                add_state = stream_state

            if add_state:
                result.append((display, item))

        return result


    # Events Dropdown list
    select_events = widgets.Dropdown(
        options=create_event_list_for_dropdown(event_list, streamable_events),
        disabled=False,
        value=None,
    )

    print()
    print('➕➕➕ 🔰 Select from Event List ➕➕➕')
    print()
    display(widgets.Label(value="🌈 Event List: "))
    display(select_events)

    # Function called when a Dropdown is selected
    def on_select_events_handler(change):
        """
        Handle event selection from the dropdown.

        This function is called when an event is selected from the dropdown list. It takes the selected event item,
        maintains information about the selected event, and triggers the display of an end message.

        Args:
            change (dict): A dictionary containing information about the event selection.

        Returns:
            None
        """
        selected_item = change['new']

        # Maintain information about selected devices
        set_selected_event(selected_item)

        # Process end message display
        print_end_msg()

    # Handler Setup
    select_events.observe(on_select_events_handler, names='value')


#
# --------------- Execution ---------------
#


# Process start message display
print_start_msg()

# Get the SORACOM API client.
soracom_api_client = get_soracom_api_client()

# Get the selected camera id.
selected_camera_id = get_selected_camera_id()

# Get the selected event time.
selected_event_time = get_selected_event_time()

# Get a list of events
event_list = get_event_list(soracom_api_client, selected_camera_id, 100, selected_event_time.get_from_ms(), selected_event_time.get_to_ms())

print()
print('# 🎰 event list = ', event_list)
print('# Number of events = ', len(event_list))

# Show the UI to select an event.
show_event_list_widgets(event_list, streamable_events)

# Select the first event in the list
if len(event_list) == 1:
    set_selected_event(event_list[0])

# Process end message display
print_end_msg()


In [ ]:
# @title ステップ 9: 選択したイベントの録画映像をストリーミング再生する

#
# --------------- Definition ---------------
#

from google.colab.output import eval_js
from IPython.display import HTML


# Get the playback range from the event.
def get_event_playback_range(selected_event: dict) -> ExportTimeInfo:
    """
    Get the playback range from the selected event.

    This function retrieves the playback range (start and end times) from the selected event's information
    and returns it as an ExportTimeInfo object.

    Args:
        selected_event (dict): The selected event's information as a dictionary.

    Returns:
        ExportTimeInfo: An ExportTimeInfo object representing the playback range.

    """
    print()
    print('# Event time = ', datetime.fromtimestamp(selected_event['time'] / 1000, JST))

    # Retrieve the start and end times from the selected event.
    start = 0
    if selected_event.get('eventInfo'):
        if selected_event.get('eventInfo').get('atomEventV1'):
            if selected_event.get('eventInfo').get('atomEventV1').get('startTime'):
                start = selected_event.get('eventInfo').get(
                    'atomEventV1').get('startTime')

    end = 0
    if selected_event.get('eventInfo'):
        if selected_event.get('eventInfo').get('atomEventV1'):
            if selected_event.get('eventInfo').get('atomEventV1').get('endTime'):
                end = selected_event.get('eventInfo').get(
                    'atomEventV1').get('endTime')

    stream_time_info: ExportTimeInfo = ExportTimeInfo(None, None)
    if start > 0 and end > 0:
        stream_time_info = ExportTimeInfo(datetime.fromtimestamp(
            start / 1000, JST), datetime.fromtimestamp(end / 1000, JST))

        print('# Event start time = ', stream_time_info.time_from)
        print('# Event end time = ', stream_time_info.time_to)
        print('# Event video duration (seconds) = ', stream_time_info.get_export_sec())

    return stream_time_info


def show_stream_video_widgets(soracom_api_client: APIClient, camera_id: str, stream_time_info: ExportTimeInfo):
    """
    Display a streaming video player and related widgets.

    Args:
        soracom_api_client (APIClient): An instance of the Soracom API client.
        camera_id (str): The camera ID for which to display the streaming video.
        stream_time_info (ExportTimeInfo): An object containing time information for video export.

    Returns:
        None
    """

    def get_stream_video_url(soracom_api_client: APIClient, camera_id: str, time_from: int = None, time_to: int = None) -> tuple[str, datetime]:
        """
        Get the streaming video URL and expiry time for a specific camera and time range.

        Args:
            soracom_api_client (APIClient): An instance of the Soracom API client.
            camera_id (str): The camera ID for which to retrieve the streaming video.
            time_from (int): The start time for the video export in milliseconds (optional).
            time_to (int): The end time for the video export in milliseconds (optional).

        Returns:
            tuple[str, datetime]: A tuple containing the video URL (str) and the expiry time (datetime).
        """
        stream_info = None
        api_result = soracom_api_client.get_sora_cam_streaming_video(camera_id, time_from, time_to)

        if api_result.check_success():
            stream_info = api_result.response

        video_url = stream_info['playList'][0]['url']
        expiry_time = datetime.fromtimestamp(stream_info['expiryTime'] / 1000, JST)

        print()
        print('# 📺 Video Information = ', stream_info)
        print('# video_url = ', video_url)
        print('# expiry_time = ', expiry_time)

        # Display remaining time for video export
        get_remaining_export_time(soracom_api_client, camera_id)

        return video_url, expiry_time


    def display_stream_player(url: str, expiry_time: int):
        """
        Display a video player for streaming video playback.

        Args:
            url (str): The URL of the streaming video.
            expiry_time (int): The expiry time of the streaming video.
        """

        video_html = HTML('''
            <style>
                video {
                    width: 960px;
                    height: 540px;
                }
            </style>
            <div id="url"></div>
            <div id="time"></div>
            <div>
                <video id="videoPlayer" controls></video>
            </div>
            <script src="//cdn.dashjs.org/latest/dash.all.debug.js"></script>
            <script>
                function init_video(url, time){
                var player = dashjs.MediaPlayer().create();
                player.initialize(document.querySelector("#videoPlayer"), url, true);
                document.getElementById("url").innerHTML = '<p> 🌏 url = ' + url + '</p>';
                document.getElementById("time").innerHTML = '<p>⏳ expiry_time = <b>' + time + '</b></p>';
                }
                init_video("%s", "%s")
            </script>
        ''' % (url, expiry_time))

        print()
        display(video_html)


    # Reload button
    stream_reload_button = widgets.Button(
        description='Reload video',
        disabled=False,
        button_style='info',  # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Reload video',
        icon='rotate-right'  # (FontAwesome names without the `fa-` prefix)
    )

    stream_type_label = widgets.Label(value='ℹ️ Real-time video is playing.')

    # Real time or no range?
    if stream_time_info.valid_vars():
        stream_type_label.value = 'ℹ️ The specified time period is played back.（{} - {}）'.format(
            stream_time_info.time_from, stream_time_info.time_to)

    # Initial display of video
    video_url, expiry_time = get_stream_video_url(soracom_api_client, camera_id, stream_time_info.get_from_ms(), stream_time_info.get_to_ms())

    # Display HTML for streaming video viewing
    display_stream_player(video_url, expiry_time)
    print()
    display(widgets.HBox([stream_reload_button,
            widgets.Label(value='　'), stream_type_label]))


    # Function called when a button is pressed
    def on_stream_reload_button_handler(change):
        """
        Handle the event when the streaming video reload button is pressed.

        Args:
            change: The change event object.
        """

        # Video reload process
        video_url, expiry_time = get_stream_video_url(soracom_api_client, camera_id, stream_time_info.get_from_ms(), stream_time_info.get_to_ms())

        # Reflects values to the Javascript
        eval_js('init_video("{}", "{}")'.format(video_url, expiry_time))

        # Process end message display
        print_end_msg()

    # Handler Setup
    stream_reload_button.on_click(on_stream_reload_button_handler)


#
# --------------- Execution ---------------
#


# Process start message display
print_start_msg()

# Get the SORACOM API client.
soracom_api_client = get_soracom_api_client()

# Get the selected camera id.
selected_camera_id = get_selected_camera_id()

# Get the selected event.
selected_event = get_selected_event()

# Get the playback range from the event.
stream_time_info = get_event_playback_range(selected_event)

# Show the UI to play the recorded video.
show_stream_video_widgets(soracom_api_client, selected_camera_id, stream_time_info)

# Process end message display
print_end_msg()


In [ ]:
# @title ステップ 10: 選択したイベントの画像を取得する

#
# --------------- Definition ---------------
#

def download_event_image(url: str, path: str):
    """
    Download an event image from the specified URL and save it to the specified path.

    This function sends a GET request to the specified URL to retrieve the event image content and
    then saves it as an image file at the specified path.

    Args:
        url (str): The URL of the event image to download.
        path (str): The file path where the event image will be saved.

    """
    print()
    print('# 🎨 Download Event Image')
    print('# url = ', url)
    print('# image_path = ', path)

    # Request to retrieve content
    contents_result = send_get_content(url)

    if contents_result.check_success():
        # Save as an image file
        with open(path, "wb") as f:
            f.write(contents_result.content)
            print()
            print('# ✅ The event image has been saved.')


#
# --------------- Execution ---------------
#


# Process start message display
print_start_msg()

# Initialize directory to store event image file
init_dir(EVENT_IMAGE_DIR)

# Get the selected event.
selected_event = get_selected_event()

# image url
image_url = selected_event.get('eventInfo').get('atomEventV1').get('picture')
image_file_path = os.path.join(EVENT_IMAGE_DIR, extract_filename_from_URL(image_url))

# Download event image.
download_event_image(image_url, image_file_path)

# Process end message display
print_end_msg()


In [ ]:
# @title ステップ 11: 取得したイベント画像を表示して確認する


#
# --------------- Execution ---------------
#


# Process start message display
print_start_msg()

# Display images in a grid
display_grid_images(EVENT_IMAGE_DIR, '*.jpg')

# Process end message display
print_end_msg()

In [ ]:
# @title ステップ 12: 取得したイベント画像を使い OpenAI API とチャットする

#
# --------------- Definition ---------------
#

import base64


def show_openai_chat_widgets(openai_client: OpenAI, model: str, user_history: list, image_dir: str):
    """
    Display a user interface for interacting with the OpenAI chat API.

    Args:
        openai_client (OpenAI): An instance of the OpenAI client.
        model (str): The name of the AI model to use for chat interactions.
        user_history (list): A list of user chat history.
        image_dir (str): The directory containing image files for chat interactions.

    Returns:
        None
    """

    # Chat with OpenAI API
    # Reference: https://platform.openai.com/docs/api-reference/chat/create
    def create_chat_completion(openai_client: OpenAI, user_msg: str, model: str, image_path: str = None, temperature: float = None, msg_history: list[dict] = None, system_msg: str = None, token_limt: int = 4000) -> list[dict]:
        """
        Create a chat completion using the OpenAI chat API.

        Args:
            openai_client (OpenAI): An instance of the OpenAI client.
            user_msg (str): The user's message to send to the AI model.
            model (str): The name of the AI model to use for chat interactions.
            image_path (str): The path to an image file to include in the chat (optional).
            temperature (float): The temperature parameter for generating responses (optional).
            msg_history (list[dict]): A list of chat history messages (optional).
            system_msg (str): A system message to send to the AI model (optional).
            token_limt (int): The maximum number of tokens allowed in the response (default is 4000).

        Returns:
            list[dict]: A list of response messages from the AI model.
        """

        params = dict()
        params['max_tokens'] = token_limt

        if model is not None and model != '':
            params['model'] = model

        if temperature is not None:
            params['temperature'] = temperature

        messages = list()
        if system_msg is not None and system_msg != '':
            messages.append({"role": "system", "content": system_msg})

        # Create a user message
        content_item = {"role": "user", "content": user_msg}
        if image_path is not None and image_path != '':
            # Attach image file
            with open(image_path, "rb") as image_file:
                base64_image = base64.b64encode(image_file.read()).decode('utf-8')
                content_item = {"role": "user", "content": [{"type": "text", "text": user_msg},{"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}]}

        messages.append(content_item)
        params['messages'] = messages
        if msg_history is not None and len(msg_history) > 0:
            params['messages'] = msg_history + messages

        print()
        print('# Send params = ', params)

        print()
        print('# 🎨 Image = ', image_path)
        print('# 🍺 AI Model = ', model)
        print()
        print('# 👻 System message = ', system_msg)
        print('# 💸 Max tokens = ', token_limt)
        print('# 🌡️ Temperature = ', temperature)
        print()
        print('# 🧑‍💻 User message = ', user_msg)

        # Call OpenAI with configured information
        chat_completion = openai_client.chat.completions.create(**params)

        print()
        print('# Chat result = ', chat_completion)
        print()
        msg_history.extend(messages)

        # Confirm reply
        result_msg = list()
        for item in chat_completion.choices:
            print('# 🤖 Reply message = ')
            print(item.message.content.strip())

            # Keep role and content
            msg_history.append({"role": item.message.role, "content": item.message.content})
            result_msg.append({"role": item.message.role, "content": item.message.content})

        print()
        print('# 📦 Messages = ', msg_history)
        print('# 💰 Usage = ', chat_completion.usage)

        return result_msg


    # User content input field
    input_user_content = widgets.Textarea(
        value='',
        placeholder='Enter what you want to ask',
        disabled=False
    )

    # API temperature select field
    slider_api_temperature = widgets.FloatSlider(
        value=1.0,
        min=0.0,
        max=2.0,
        step=0.1,
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True,
        readout_format='.1f',
    )

    # Max tokens input slider
    slider_token_limit = widgets.IntSlider(
        value=4000,
        min=1,
        max=128000,
        step=1,
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True,
        readout_format='d',
    )

    # System content input field
    input_system_content = widgets.Textarea(
        value='',
        placeholder='Enter a message to the system',
        disabled=False
    )

    # send button
    send_button = widgets.Button(
        description='Send',
        disabled=False,
        button_style='info',  # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Send',
        icon='walkie-talkie'  # (FontAwesome names without the `fa-` prefix)
    )

    # Display images in a grid
    image_list = display_grid_images(image_dir, '*.jpg', 5, 5)
    print()
    print('# 🍺 AI Model = ', model)

    print()
    print('➕➕➕ 🔰 Enter chat message ➕➕➕')
    print()

    display(widgets.Label(value="👻 System message: "))
    display(input_system_content)
    display(widgets.Label(value="🌡️ Temperature: "))
    display(slider_api_temperature)
    display(widgets.Label(value="💸 Max tokens: "))
    display(slider_token_limit)

    print()
    display(widgets.Label(value="🧑‍💻 User message:"))
    display(input_user_content)

    print()
    display(send_button)

    # Function called when a button is pressed
    def on_send_button_handler(change):
        """
        Callback function for the send button in the OpenAI chat interface.

        This function is called when the user clicks the send button. It retrieves user input, system input,
        temperature, token limit, and image path. Then it checks for valid input, calls the OpenAI API,
        resets the input fields, and displays the API response.

        Args:
            change (dict): The change event object (not used in this function).

        Returns:
            None
        """
        user_content = input_user_content.value
        system_content = input_system_content.value
        temperature = slider_api_temperature.value
        token_limt = slider_token_limit.value

        # Check input values
        if user_content == None or user_content == '':
            print()
            print('➕➕➕ ⛔ No query has been entered. Please check your input. ➕➕➕')
            return

        # Attach the image only once at first
        file_path = None
        if len(user_history) <= 0:
            file_path = image_list[0]

        # Call OpenAI with the entered information
        create_chat_completion(openai_client, user_content, model, file_path, temperature, user_history, system_content, token_limt)

        user_content = ''
        input_user_content.value = user_content
        system_content = ''
        input_system_content.value = system_content
        slider_api_temperature.value = temperature
        temperature = None
        slider_token_limit.value = token_limt
        token_limt = None

        # Process end message display
        print_end_msg()

    # Handler Setup
    send_button.on_click(on_send_button_handler)


#
# --------------- Execution ---------------
#


# Process start message display
print_start_msg()

# Get openai client
openai_client = get_openai_client()

# Get selected openai model
selected_openai_model = get_selected_openai_model()

# Keep a record of your conversations.
user_history: list = list()

# Show the UI that allows natural language questioning of images, similar to ChatGPT.
show_openai_chat_widgets(openai_client, selected_openai_model, user_history, EVENT_IMAGE_DIR)

# Process end message display
print_end_msg()


In [ ]:
# @title ステップ 13: ローカルにダウンロードするディレクトリを選択する

#
# --------------- Definition ---------------
#

# The selected directory.
_selected_download_dir: str = None


def get_selected_download_dir() -> str:
    """
    Get the selected download directory path.

    Returns:
        str: The selected download directory path.
    """
    return _selected_download_dir

def set_selected_download_dir(download_dir: str):
    """
    Set the selected download directory path and perform additional actions.

    Args:
        download_dir (str): The new download directory path to set.

    Returns:
        None
    """
    global _selected_download_dir
    _selected_download_dir = download_dir

    print()
    print('# 🎯 Selected Dir = ', _selected_download_dir)

    # Get a list of files in a dir
    files = glob.glob(os.path.join(_selected_download_dir, '*'))
    print('# 📄 File List = ', files)

    if _selected_download_dir == USER_CONTENT_DIR:
        print()
        print('# 🧨 The root of the content folder is specified. Compression and download will take some time. Please check before executing.')


def show_download_dir_widgets(dir_list: str):
    """
    Display a widget for selecting a directory from a list of directories.

    Args:
        dir_list (str): A list of directory options to display in the dropdown.

    Returns:
        None
    """

    # Dir Dropdown list
    select_dirs = widgets.Dropdown(
        options=dir_list,
        disabled=False,
        value=None,
    )
    print()
    print('➕➕➕ 🔰 Please select one from the list of directories below. ➕➕➕')
    print()

    display(widgets.Label(value="🗂️ Directories:"))
    display(select_dirs)

    # Function called when a Dropdown is selected
    def on_select_dirs_handler(change):
        """
        Handle the selection of a directory from a dropdown list.

        This function is called when a directory is selected from a dropdown list of directories. It prints
        information about the selected directory, including the list of files within that directory. If the
        root content directory is selected, it displays a warning message about the potential time required
        for compression and download.

        Args:
            change (dict): A dictionary containing the change event information.

        Returns:
            None
        """
        selected_item = change['new']

        # Hold the selected download target directory.
        set_selected_download_dir(selected_item)

        # Process end message display
        print_end_msg()


    # Handler Setup
    select_dirs.observe(on_select_dirs_handler, names='value')


#
# --------------- Execution ---------------
#


# Process start message display
print_start_msg()

path = USER_CONTENT_DIR
name = '**'

print()
print('# path = ', path)
print('# name = ', name)

# Get Dir list
file_list = glob.glob(os.path.join(path, name), recursive=True)
dir_list = [f for f in file_list if os.path.isdir(f)]
dir_list.sort()

print()
print('# 🗂️ Directories = ', dir_list)

# Display the UI to select a directory.
show_download_dir_widgets(dir_list)

# Select the first dir in the list
if len(dir_list) == 1:
    set_selected_download_dir(dir_list[0])

# Process end message display
print_end_msg()


In [ ]:
# @title ステップ 14: 選択したディレクトリをローカルにダウンロードする


#
# --------------- Execution ---------------
#


# Process start message display
print_start_msg()

# Get the selected download dir
selected_download_dir = get_selected_download_dir()

# Download the whole specified directory
download_dir_locally(selected_download_dir, ZIP_WORK_DIR)

# Process end message display
print_end_msg()


In [ ]:
# @title ステップ 15: SORACOM API からログアウトする


#
# --------------- Execution ---------------
#


# Process start message display
print_start_msg()

# Logout and Delete of SORACOM API client
logout_delete_soracom_api_client()

# Delete an instance of OpenAI
delete_openai_client()

# Process end message display
print_end_msg()


## [付録] : クラウドに保存された録画映像から静止画をエクスポートして利用する

---

ここまでのサンプルコードでは、イベント一覧から特定のイベントの画像を取得し、それを利用する方法を紹介しました。
しかし、定点での観測を行っている場合やイベント検出機能がオフになっている状況では、必ずしもイベントが記録されているとは限りません。

<br>

そこで、この **[付録]** では「クラウドに保存された録画映像から静止画をエクスポートする」サンプルを紹介します。このサンプルを用いれば、指定した日時の画像を自由に取り出して、OpenAI API で利用できます。

<br>

**[付録]** の手順に進む前に、[ステップ 6: カメラを 1 台選択する](#scrollTo=uWrfelCgof0j) までの手順を完了させる必要があります。

**[付録]** のすべてのステップを完了した後、[ステップ 12: 取得したイベント画像を使い OpenAI API とチャットする](#scrollTo=eOeDNNs-CXcM) を実行することで、エクスポートした静止画を OpenAI API で利用できます。


In [ ]:
# @title [付録] ステップ １: 録画状態の確認範囲を指定する

#
# --------------- Definition ---------------
#

# Define variables to hold values
_selected_data_time: ExportTimeInfo = ExportTimeInfo(None, None)


def get_selected_data_time() -> ExportTimeInfo:
    """
    Get the selected data time range.

    Returns:
        ExportTimeInfo: An object representing the selected time range for data retrieval.
    """
    return _selected_data_time

def set_selected_data_time(time_from: datetime, time_to: datetime):
    """
    Set the selected data time range and display related information.

    Args:
        time_from (datetime): The start date and time for data retrieval.
        time_to (datetime): The end date and time for data retrieval.

    Returns:
        None
    """
    global _selected_data_time
    _selected_data_time.time_from = time_from
    _selected_data_time.time_to = time_to

    print()
    if _selected_data_time.valid_vars():
        print('# 🗓️ Set the start time and end time for retrieving the recording status. 🆗')
        print()
        print('# Start date time = ', _selected_data_time.time_from)
        print('# End date time = ', _selected_data_time.time_to)
        print('# Export range = ', _selected_data_time.time_to - _selected_data_time.time_from)
    else:
        print('# 🔋 Set the retrieve all recording status. 🆗')


def show_data_time_range_widgets():
    """
    Display widgets for setting the period for retrieving data.

    This function displays a user interface that allows users to set the start and end times for data retrieval.
    Users can also choose to retrieve data for the full term. It provides options for selecting dates and times,
    and when the user confirms the time range, it calls the appropriate handler function for processing.

    Returns:
        None
    """

    # Prepare a list since time picker is not available
    hours = [i for i in range(24)]
    minutes = [i for i in range(60)]
    seconds = [i for i in range(60)]
    jst_now = datetime.now(JST)
    one_hour_ago = jst_now - timedelta(hours=1)

    # Start date
    data_start_date = widgets.DatePicker(
        description='Start date:',
        value=one_hour_ago.date(),
        disabled=False
    )

    # Start time
    data_start_hours = widgets.Dropdown(
        options=hours,
        value=one_hour_ago.hour,
        description='Start time:',
        disabled=False,
    )

    data_start_minutes = widgets.Dropdown(
        options=minutes,
        value=one_hour_ago.minute,
        disabled=False,
    )

    data_start_seconds = widgets.Dropdown(
        options=seconds,
        value=one_hour_ago.second,
        disabled=False,
    )

    # End date
    data_end_date = widgets.DatePicker(
        description='End date:',
        value=jst_now.date(),
        disabled=False
    )

    # End time
    data_end_hours = widgets.Dropdown(
        options=hours,
        value=jst_now.hour,
        description='End time:',
        disabled=False,
    )

    data_end_minutes = widgets.Dropdown(
        options=minutes,
        value=jst_now.minute,
        disabled=False,
    )

    data_end_seconds = widgets.Dropdown(
        options=seconds,
        value=jst_now.second,
        disabled=False,
    )

    data_time_button = widgets.Button(
        description='Time setting',
        disabled=False,
        button_style='info',  # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Time setting',
        icon='clock'  # (FontAwesome names without the `fa-` prefix)
    )

    data_full_button = widgets.Button(
        description='Full term',
        disabled=False,
        button_style='success',  # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Full term',
        icon='gas-pump'  # (FontAwesome names without the `fa-` prefix)
    )

    # Display the time setting UI
    print()
    print('➕➕➕ 🔰 Set the period for retrieving data. ➕➕➕')
    print()
    display(widgets.HBox([data_start_date, data_start_hours, widgets.Label(
        value='：'), data_start_minutes, widgets.Label(value='：'), data_start_seconds]))
    display(widgets.HBox([data_end_date, data_end_hours, widgets.Label(
        value='：'), data_end_minutes, widgets.Label(value='：'), data_end_seconds]))
    print()
    display(widgets.HBox(
        [data_time_button, widgets.Label(value='  '), data_full_button]))

    # Function called when a button is pressed
    def on_data_time_button_handler(change):
        """
        Handle the button click event for setting data retrieval time.

        This function is called when the "Set Time" button is clicked to specify the start and end times for data retrieval.
        It retrieves the selected date and time values from the widgets, formats them into ISO datetime strings,
        and then converts them to datetime objects. It also performs checks on the entered time and displays
        information about the selected time range. Finally, it processes the end message display.

        Args:
            change (dict): A dictionary containing the change event information.

        Returns:
            None
        """
        start_date = data_start_date.value
        start_hour = data_start_hours.value
        start_minute = data_start_minutes.value
        start_second = data_start_seconds.value

        # ISO format for start time
        # '2018-12-31T05:00:30.001000+09:00'
        start_datetime_iso = '{}T{:0=2}:{:0=2}:{:0=2}.000000+09:00'.format(
            start_date, start_hour, start_minute, start_second)
        start_datetime = datetime.fromisoformat(start_datetime_iso)

        end_date = data_end_date.value
        end_hour = data_end_hours.value
        end_minute = data_end_minutes.value
        end_second = data_end_seconds.value

        # ISO format for end time
        # '2018-12-31T05:00:30.001000+09:00'
        end_datetime_iso = '{}T{:0=2}:{:0=2}:{:0=2}.000000+09:00'.format(
            end_date, end_hour, end_minute, end_second)
        end_datetime = datetime.fromisoformat(end_datetime_iso)

        # Check the entered time
        if not ExportTimeInfo(start_datetime, end_datetime).check_time_limits(False):
            return

        # Holds the entered value
        set_selected_data_time(start_datetime, end_datetime)

        # Process end message display
        print_end_msg()

    # Function called when a button is pressed
    def on_data_full_button_handler(change):
        """
        Handle the button click event for retrieving all data.

        This function is called when the "Full term" button is clicked. It sets the data retrieval time
        to retrieve all available data (full time). It then displays a confirmation message and processes
        the end message display.

        Args:
            change (dict): A dictionary containing the change event information.

        Returns:
            None
        """
        # It is treated as full time if no time is specified.
        set_selected_data_time(None, None)

        # Process end message display
        print_end_msg()


    # Handler Setup
    data_time_button.on_click(on_data_time_button_handler)

    # Handler Setup
    data_full_button.on_click(on_data_full_button_handler)


#
# --------------- Execution ---------------
#


# Process start message display
print_start_msg()

# Show the UI for specifying a time range.
show_data_time_range_widgets()

# Process end message display
print_end_msg()


In [ ]:
# @title [付録] ステップ 2: 指定した範囲の録画状態を取得する

#
# --------------- Definition ---------------
#

from pprint import pprint

# Define variables to hold values
_selected_record_status_list: list = None
_selected_event_status_list: list = None


def get_selected_record_status_list() -> list:
    """
    Get the selected list of recording status data.

    Returns:
        list: A list of selected recording status data.
    """
    return _selected_record_status_list

def get_selected_event_status_list() -> list:
    """
    Get the selected list of event status data.

    Returns:
        list: A list of selected event status data.
    """
    return _selected_event_status_list

def set_selected_record_status_list(record_status_list: list):
    """
    Set the selected list of recording status data.

    Args:
        record_status_list (list): A list of recording status data to be selected and stored.

    Returns:
        None
    """
    global _selected_record_status_list
    _selected_record_status_list = record_status_list

def set_selected_event_status_list(event_status_list: list):
    """
    Set the selected list of event status data.

    Args:
        event_status_list (list): A list of event status data to be selected and stored.

    Returns:
        None
    """
    global _selected_event_status_list
    _selected_event_status_list = event_status_list


# Retrieve a list of recording associated with the camera.
def get_recording_list(soracom_api_client: APIClient, camera_id: str, time_from: int = None, time_to: int = None, last_key: str = None) -> tuple[list, list]:
    """
    Retrieve a list of recordings and events associated with a camera.

    This function queries the SORACOM API to retrieve a list of recording and event data associated with a specific camera.
    It allows for specifying a time range and pagination using the `last_key` parameter. The function recursively retrieves
    additional data if pagination is required. The retrieved recordings and events are returned as two separate lists.

    Args:
        soracom_api_client (APIClient): An instance of the SORACOM API client.
        camera_id (str): The ID of the camera for which to retrieve recordings and events.
        time_from (int): The start time for the retrieval in milliseconds since the epoch (optional).
        time_to (int): The end time for the retrieval in milliseconds since the epoch (optional).
        last_key (str): A pagination key to retrieve additional data (optional).

    Returns:
        tuple[list, list]: A tuple containing two lists: the first list contains recording data, and the second list
        contains event data.
    """

    result_records = list()
    result_events = list()

    # Retrieve the recording status.
    api_result = soracom_api_client.list_sora_cam_recordings_and_events(camera_id, time_from, time_to, last_key)

    if api_result.check_success():
        # HTTP response is 200, but body may be missing
        if api_result.response is not None:
            if api_result.response.get('records'):
                result_records.extend(api_result.response.get('records'))
            if api_result.response.get('events'):
                result_events.extend(api_result.response.get('events'))

        # Is there more to the list?
        if api_result.additions.get('headers'):
            if api_result.additions.get('headers').get('X-Soracom-Next-Key'):
                next_key = api_result.additions.get('headers').get('X-Soracom-Next-Key')
                print()
                print('# ⏭ next_key = ', next_key)

                record_list, event_list = get_recording_list(soracom_api_client, camera_id, time_from, time_to, next_key)
                result_records.extend(record_list)
                result_events.extend(event_list)

    return result_records, result_events


# Edit the date and time for use in a DataFrame.
def edit_recording_list(record_list: list) -> list:
    """
    Edit the date and time information in a list of recording data for use in a DataFrame.

    This function takes a list of recording data and edits the date and time information for each recording item.
    It converts the timestamp values (in milliseconds since the epoch) for 'startTime' and 'endTime' to datetime
    objects in the local time zone (JST) with microsecond precision. If 'endTime' is missing, it is set to 'startTime'.
    The edited recording data is returned as a list of dictionaries.

    Args:
        record_list (list): A list of recording data where each item is represented as a dictionary.

    Returns:
        list: A list of dictionaries containing edited recording data with datetime values for 'startTime' and 'endTime'.
    """

    edited_records = list()

    for item in record_list:
        item_val = None
        item_dict = dict()

        if item.get('startTime'):
            item_val = item.get('startTime')
            date_time = datetime.fromtimestamp(item_val / 1000, JST)
            date_time = date_time.replace(tzinfo=None).replace(microsecond=0)
            item_dict['startTime'] = date_time

        if item.get('endTime'):
            item_val = item.get('endTime')
            date_time = datetime.fromtimestamp(item_val / 1000, JST)
            date_time = date_time.replace(tzinfo=None).replace(microsecond=0)
            item_dict['endTime'] = date_time

        if not item_dict.get('endTime'):
            if item_dict.get('startTime'):
                item_dict['endTime'] = item_dict.get('startTime')

        if item.get('type'):
            item_dict['type'] = item.get('type')

        edited_records.append(item_dict)

    return edited_records


#
# --------------- Execution ---------------
#


# Process start message display
print_start_msg()

# Get the SORACOM API client.
soracom_api_client = get_soracom_api_client()

# Get the selected camera id.
selected_camera_id = get_selected_camera_id()

# Get the selected data time range.
selected_data_time = get_selected_data_time()

# Get the recording status
record_list, event_list = get_recording_list(soracom_api_client, selected_camera_id, selected_data_time.get_from_ms(), selected_data_time.get_to_ms())

print()
print('# 📼 record list = ', record_list)
print('# 📜 Number of All record list = ', len(record_list))

# Edit the recording status.
edited_record_list = edit_recording_list(record_list)
set_selected_record_status_list(edited_record_list)

print('# edited record list =', edited_record_list)
pprint(edited_record_list)

print()
print('# 🎰 event list = ', event_list)
print('# 📜 Number of All event list = ', len(event_list))

# Edit the event status.
edited_event_list = edit_recording_list(event_list)
set_selected_event_status_list(edited_event_list)

print('# edited event list =', edited_event_list)
pprint(edited_event_list)

# Process end message display
print_end_msg()


In [ ]:
# @title [付録] ステップ 3: 取得した録画状態をグラフ表示する

#
# --------------- Definition ---------------
#

import math

import matplotlib.dates as mdates
import matplotlib.ticker as ticker
import pandas as pd


# Select the appropriate locator and formatter
def select_locator_and_formatter(df: pd.DataFrame, max_ticks: int = 50) -> tuple[ticker.Locator, str]:
    """
    Select the appropriate Locator and Formatter for the X-axis based on the date range of the data frame.

    Args:
        df (pd.DataFrame): The DataFrame containing the data to be plotted.
        max_ticks (int, optional): The maximum number of ticks (labels) to appear on the X-axis. Defaults to 50.

    Returns:
        Tuple[ticker.Locator, str]: A tuple containing the selected Locator and Formatter for the X-axis.
    """

    # Analyze date range of data frame
    date_range = df['endTime'].max() - df['startTime'].min()

    # Select the appropriate Locator based on the range
    range_minutes = date_range.total_seconds() / 60
    locator = mdates.MinuteLocator(interval=1)
    formatter = '%y-%m-%d %H:%M'

    # Set no more than max_ticks to appear on the X axis
    if range_minutes >= max_ticks:
        interval = abs(math.ceil(range_minutes/max_ticks))
        # Adjust to a range of max_ticks
        locator = mdates.MinuteLocator(interval=interval)

    # Adjust range in seconds if less than 5 minute
    if range_minutes <= 5:
        range_seconds = date_range.total_seconds()
        locator = mdates.SecondLocator(interval=1)
        formatter = '%y-%m-%d %H:%M:%S'

        if range_seconds >= max_ticks:
            interval = abs(math.ceil(range_seconds/max_ticks))
            locator = mdates.SecondLocator(interval=interval)

    return locator, formatter


def show_recording_status_graph(data: list, width: int = 15, height: int = 5):
    """
    Create a timeline graph of records based on the provided data.

    Args:
        data (list): A list of data representing records with start and end times.
        width (int, optional): The width of the graph in inches. Defaults to 15.
        height (int, optional): The height of the graph in inches. Defaults to 5.

    Returns:
        None
    """

    # Convert it into a DataFrame.
    df = pd.DataFrame(data)

    # Create a figure for data
    fig, ax = plt.subplots(figsize=(width, height))

    # Prepare the necessary information for display.
    title = 'Timeline of Records'
    ylabel = 'Records'
    marker = 'o'
    color = 'orange'

    # Check if a specific column name exists.
    column_name = 'type'
    label_name = 'motion'
    if column_name in df.columns:
        # The event data is the subject for visualization.
        title = 'Timeline of Events'
        ylabel = 'Events'
        marker = 'x'
        color = 'blue'

    # Plot data
    for row in df.itertuples():
        label = getattr(row, 'type', None)

        if label is not None:
            if label != label_name:
                color = None

        ax.plot([row.startTime, row.endTime], [0, 0],
                marker=marker, color=color, label=label)

    locator, formatter = select_locator_and_formatter(df)
    ax.xaxis.set_major_locator(locator)
    ax.xaxis.set_major_formatter(mdates.DateFormatter(formatter))

    # Erase Y-axis numeric display
    ax.set_yticks([])
    ax.set_yticklabels([])

    plt.xticks(rotation=45)
    plt.xlabel('Time')
    plt.ylabel(ylabel)
    plt.title(title)
    plt.tight_layout()

    # Edit the legend.
    if column_name in df.columns:
        handles, labels = plt.gca().get_legend_handles_labels()
        by_label = dict(zip(labels, handles))
        plt.legend(by_label.values(), by_label.keys())

    # Show the plots
    plt.show()


#
# --------------- Execution ---------------
#


# Process start message display
print_start_msg()

# Get the list of recording data.
record_status_list = get_selected_record_status_list()

# Recording status of the selected camera.
# Process records data
show_recording_status_graph(record_status_list)

# Get the list of event recording data.
event_status_list = get_selected_event_status_list()

# Event recording status of selected camera.
# Process events data
show_recording_status_graph(event_status_list)

# Process end message display
print_end_msg()


In [ ]:
# @title [付録] ステップ 4: 録画映像のストリーミング再生範囲を指定する

#
# --------------- Definition ---------------
#

# Define variables to hold values
_selected_stream_time: ExportTimeInfo = ExportTimeInfo(None, None)


def get_selected_stream_time() -> ExportTimeInfo:
    """
    Get the selected streaming playback time range.

    Returns:
        ExportTimeInfo: An object representing the selected start and end times for streaming playback.
    """
    return _selected_stream_time

def set_selected_stream_time(time_from: datetime, time_to: datetime):
    """
    Set the selected streaming playback time range.

    Args:
        time_from (datetime): The start time for streaming playback.
        time_to (datetime): The end time for streaming playback.

    Returns:
        None
    """
    global _selected_stream_time
    _selected_stream_time.time_from = time_from
    _selected_stream_time.time_to = time_to

    print()
    if _selected_stream_time.valid_vars():
        print('# 📼 Set the start and end times for streaming playback. 🆗')

        print()
        print('# Start date time = ', _selected_stream_time.time_from)
        print('# End date time = ', _selected_stream_time.time_to)
        print('# Export range = ', _selected_stream_time.time_to - _selected_stream_time.time_from)

        print()
        print('# Performing a playback will consume the {} seconds export time.'.format(_selected_stream_time.get_export_sec()))
    else:
        print('# 📹 Set the real time for streaming playback. 🆗')
        print()
        print('# Performing a playback will consume the maximum time for the unit export.')


def show_stream_time_range_widgets(soracom_api_client: APIClient, camera_id: str):
    """
    Display widgets for setting the time range for streaming playback.

    This function displays a user interface with widgets for setting the start and end times for streaming playback.
    Users can specify a custom time range or choose real-time streaming. It also provides buttons to confirm the
    selected time range or choose real-time streaming.

    Args:
        soracom_api_client (APIClient): The SORACOM API client.
        camera_id (str): The ID of the camera.

    Returns:
        None
    """

    # Prepare a list since time picker is not available
    hours = [i for i in range(24)]
    minutes = [i for i in range(60)]
    seconds = [i for i in range(60)]
    jst_now = datetime.now(JST)
    one_minute_ago = jst_now - timedelta(minutes=1)

    # Start date
    stream_start_date = widgets.DatePicker(
        description='Start date:',
        value=one_minute_ago.date(),
        disabled=False
    )

    # Start time
    stream_start_hours = widgets.Dropdown(
        options=hours,
        value=one_minute_ago.hour,
        description='Start time:',
        disabled=False,
    )

    stream_start_minutes = widgets.Dropdown(
        options=minutes,
        value=one_minute_ago.minute,
        disabled=False,
    )

    stream_start_seconds = widgets.Dropdown(
        options=seconds,
        value=one_minute_ago.second,
        disabled=False,
    )

    # End date
    stream_end_date = widgets.DatePicker(
        description='End date:',
        value=jst_now.date(),
        disabled=False
    )

    # End time
    stream_end_hours = widgets.Dropdown(
        options=hours,
        value=jst_now.hour,
        description='End time:',
        disabled=False,
    )

    stream_end_minutes = widgets.Dropdown(
        options=minutes,
        value=jst_now.minute,
        disabled=False,
    )

    stream_end_seconds = widgets.Dropdown(
        options=seconds,
        value=jst_now.second,
        disabled=False,
    )

    stream_time_button = widgets.Button(
        description='Time setting',
        disabled=False,
        button_style='info',  # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Time setting',
        icon='clock'  # (FontAwesome names without the `fa-` prefix)
    )

    stream_real_button = widgets.Button(
        description='Real Time',
        disabled=False,
        button_style='success',  # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Real Time',
        icon='rocket-launch'  # (FontAwesome names without the `fa-` prefix)
    )

    # Display the time setting UI
    print()
    print('➕➕➕ 🔰 Set the time for streaming playback. ➕➕➕')
    print()
    display(widgets.HBox([stream_start_date, stream_start_hours, widgets.Label(
        value='：'), stream_start_minutes, widgets.Label(value='：'), stream_start_seconds]))
    display(widgets.HBox([stream_end_date, stream_end_hours, widgets.Label(
        value='：'), stream_end_minutes, widgets.Label(value='：'), stream_end_seconds]))
    print()
    display(widgets.HBox(
        [stream_time_button, widgets.Label(value='  '), stream_real_button]))


    # Function called when a button is pressed
    def on_stream_time_button_handler(change):
        """
        Handle the button click event for setting a specific time range for streaming playback.

        Args:
            change: The change event object.

        Returns:
            None
        """

        start_date = stream_start_date.value
        start_hour = stream_start_hours.value
        start_minute = stream_start_minutes.value
        start_second = stream_start_seconds.value

        # ISO format for start time
        # '2018-12-31T05:00:30.001000+09:00'
        start_datetime_iso = '{}T{:0=2}:{:0=2}:{:0=2}.000000+09:00'.format(
            start_date, start_hour, start_minute, start_second)
        start_datetime = datetime.fromisoformat(start_datetime_iso)

        end_date = stream_end_date.value
        end_hour = stream_end_hours.value
        end_minute = stream_end_minutes.value
        end_second = stream_end_seconds.value

        # ISO format for end time
        # '2018-12-31T05:00:30.001000+09:00'
        end_datetime_iso = '{}T{:0=2}:{:0=2}:{:0=2}.000000+09:00'.format(
            end_date, end_hour, end_minute, end_second)
        end_datetime = datetime.fromisoformat(end_datetime_iso)

        # Check the entered time
        if not ExportTimeInfo(start_datetime, end_datetime).check_time_limits():
            return

        # Holds the entered value
        set_selected_stream_time(start_datetime, end_datetime)

        # Display remaining time for video export
        get_remaining_export_time(soracom_api_client, camera_id)

        # Process end message display
        print_end_msg()

    # Function called when a button is pressed
    def on_stream_real_button_handler(change):
        """
        Handle the button click event for real-time streaming playback.

        Args:
            change: The change event object.

        Returns:
            None
        """

        # It is treated as real time if no time is specified.
        set_selected_stream_time(None, None)

        # Display remaining time for video export
        get_remaining_export_time(soracom_api_client, camera_id)

        # Process end message display
        print_end_msg()

    # Handler Setup
    stream_time_button.on_click(on_stream_time_button_handler)

    # Handler Setup
    stream_real_button.on_click(on_stream_real_button_handler)


#
# --------------- Execution ---------------
#


# Process start message display
print_start_msg()

# Get the SORACOM API client.
soracom_api_client = get_soracom_api_client()

# Get the selected camera id.
selected_camera_id = get_selected_camera_id()

# Show the UI for specifying the streaming playback range.
show_stream_time_range_widgets(soracom_api_client, selected_camera_id)

# Process end message display
print_end_msg()


In [ ]:
# @title [付録] ステップ 5: 指定した範囲の録画映像をストリーミング再生する

#
# --------------- Definition ---------------
#

from google.colab.output import eval_js
from IPython.display import HTML


def show_stream_video_widgets(soracom_api_client: APIClient, camera_id: str, stream_time_info: ExportTimeInfo):
    """
    Display a streaming video player and related widgets.

    Args:
        soracom_api_client (APIClient): An instance of the Soracom API client.
        camera_id (str): The camera ID for which to display the streaming video.
        stream_time_info (ExportTimeInfo): An object containing time information for video export.

    Returns:
        None
    """

    def get_stream_video_url(soracom_api_client: APIClient, camera_id: str, time_from: int = None, time_to: int = None) -> tuple[str, datetime]:
        """
        Get the streaming video URL and expiry time for a specific camera and time range.

        Args:
            soracom_api_client (APIClient): An instance of the Soracom API client.
            camera_id (str): The camera ID for which to retrieve the streaming video.
            time_from (int): The start time for the video export in milliseconds (optional).
            time_to (int): The end time for the video export in milliseconds (optional).

        Returns:
            tuple[str, datetime]: A tuple containing the video URL (str) and the expiry time (datetime).
        """
        stream_info = None
        api_result = soracom_api_client.get_sora_cam_streaming_video(camera_id, time_from, time_to)

        if api_result.check_success():
            stream_info = api_result.response

        video_url = stream_info['playList'][0]['url']
        expiry_time = datetime.fromtimestamp(stream_info['expiryTime'] / 1000, JST)

        print()
        print('# 📺 Video Information = ', stream_info)
        print('# video_url = ', video_url)
        print('# expiry_time = ', expiry_time)

        # Display remaining time for video export
        get_remaining_export_time(soracom_api_client, camera_id)

        return video_url, expiry_time


    def display_stream_player(url: str, expiry_time: int):
        """
        Display a video player for streaming video playback.

        Args:
            url (str): The URL of the streaming video.
            expiry_time (int): The expiry time of the streaming video.
        """

        video_html = HTML('''
            <style>
                video {
                    width: 960px;
                    height: 540px;
                }
            </style>
            <div id="url"></div>
            <div id="time"></div>
            <div>
                <video id="videoPlayer" controls></video>
            </div>
            <script src="//cdn.dashjs.org/latest/dash.all.debug.js"></script>
            <script>
                function init_video(url, time){
                var player = dashjs.MediaPlayer().create();
                player.initialize(document.querySelector("#videoPlayer"), url, true);
                document.getElementById("url").innerHTML = '<p> 🌏 url = ' + url + '</p>';
                document.getElementById("time").innerHTML = '<p>⏳ expiry_time = <b>' + time + '</b></p>';
                }
                init_video("%s", "%s")
            </script>
        ''' % (url, expiry_time))

        print()
        display(video_html)


    # Reload button
    stream_reload_button = widgets.Button(
        description='Reload video',
        disabled=False,
        button_style='info',  # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Reload video',
        icon='rotate-right'  # (FontAwesome names without the `fa-` prefix)
    )

    stream_type_label = widgets.Label(value='ℹ️ Real-time video is playing.')

    # Real time or no range?
    if stream_time_info.valid_vars():
        stream_type_label.value = 'ℹ️ The specified time period is played back.（{} - {}）'.format(
            stream_time_info.time_from, stream_time_info.time_to)

    # Initial display of video
    video_url, expiry_time = get_stream_video_url(soracom_api_client, camera_id, stream_time_info.get_from_ms(), stream_time_info.get_to_ms())

    # Display HTML for streaming video viewing
    display_stream_player(video_url, expiry_time)
    print()
    display(widgets.HBox([stream_reload_button,
            widgets.Label(value='　'), stream_type_label]))


    # Function called when a button is pressed
    def on_stream_reload_button_handler(change):
        """
        Handle the event when the streaming video reload button is pressed.

        Args:
            change: The change event object.
        """

        # Video reload process
        video_url, expiry_time = get_stream_video_url(soracom_api_client, camera_id, stream_time_info.get_from_ms(), stream_time_info.get_to_ms())

        # Reflects values to the Javascript
        eval_js('init_video("{}", "{}")'.format(video_url, expiry_time))

        # Process end message display
        print_end_msg()

    # Handler Setup
    stream_reload_button.on_click(on_stream_reload_button_handler)


#
# --------------- Execution ---------------
#


# Process start message display
print_start_msg()

# Get the SORACOM API client.
soracom_api_client = get_soracom_api_client()

# Get the selected camera id.
selected_camera_id = get_selected_camera_id()

# Get the streaming playback time range.
stream_time_info = get_selected_stream_time()

# Show the UI to play the recorded video.
show_stream_video_widgets(soracom_api_client, selected_camera_id, stream_time_info)

# Process end message display
print_end_msg()


In [ ]:
# @title [付録] ステップ 6: 静止画ダウンロードの開始時刻を指定する

#
# --------------- Definition ---------------
#

# Define variables to hold values
_selected_image_time: datetime = None


def get_selected_image_time() -> datetime:
    """
    Get the selected start date and time for acquiring still images.

    This function retrieves and returns the datetime representing the selected start date and time for acquiring
    still images. The start date and time can be set using the `set_selected_image_time` function.

    Returns:
        datetime: The datetime representing the selected start date and time for acquiring still images.
    """
    return _selected_image_time

def set_selected_image_time(image_time: datetime):
    """
    Set the start date and time for acquiring still images.

    This function allows you to set the start date and time for acquiring still images. The specified `image_time`
    parameter should be a datetime object representing the desired start date and time for image acquisition.

    Args:
        image_time (datetime): A datetime object representing the start date and time for acquiring still images.

    Returns:
        None
    """
    global _selected_image_time
    _selected_image_time = image_time

    print()
    print('# 🖼️ The start date for acquiring still images have been set. 🆗')
    print('# Start date time = ', _selected_image_time)

    print()
    print('# When a still image is acquired will consume the number of still images time for the unit export.')


def show_image_time_widgets(soracom_api_client: APIClient, camera_id: str):
    """
    Display widgets for setting the start date and time for acquiring still images.

    This function displays user interface widgets for setting the start date and time for acquiring still images.
    It includes options for selecting the start date and time, and a button to confirm the settings.
    When the button is pressed, it checks for a future date and time, sets the selected image time, and displays
    remaining time for video export.

    Args:
        soracom_api_client (APIClient): The SORACOM API client for interacting with the SORACOM platform.
        camera_id (str): The ID of the camera associated with the still image acquisition.

    Returns:
        None
    """

    # Prepare a list since time picker is not available
    hours = [i for i in range(24)]
    minutes = [i for i in range(60)]
    seconds = [i for i in range(60)]
    jst_now = datetime.now(JST)

    # Start date
    image_start_date = widgets.DatePicker(
        description='Start date:',
        value=jst_now.date(),
        disabled=False
    )

    # Start time
    image_start_hours = widgets.Dropdown(
        options=hours,
        value=jst_now.hour,
        description='Start time:',
        disabled=False,
    )

    image_start_minutes = widgets.Dropdown(
        options=minutes,
        value=jst_now.minute,
        disabled=False,
    )

    image_start_seconds = widgets.Dropdown(
        options=seconds,
        value=jst_now.second,
        disabled=False,
    )

    image_time_setting_button = widgets.Button(
        description='Time setting',
        disabled=False,
        button_style='info',  # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Time setting',
        icon='clock'  # (FontAwesome names without the `fa-` prefix)
    )

    # Display the time setting UI
    print()
    print('➕➕➕ 🔰 Sets the start date for taking still image. ➕➕➕')
    print()
    display(widgets.HBox([image_start_date, image_start_hours, widgets.Label(
        value='：'), image_start_minutes, widgets.Label(value='：'), image_start_seconds]))
    print()
    display(image_time_setting_button)


    # Function called when a button is pressed
    def on_image_time_setting_button_handler(change):
        """
        Handler function for the button pressed to set the image capture time.

        This function is called when a button is pressed to set the image capture time. It retrieves the selected start date,
        hours, minutes, and seconds from the respective widgets. It then formats the selected values into ISO format for the
        start time. The function checks if the specified start time is in the future compared to the current time in JST timezone.
        If the start time is in the future, an error message is displayed, and the function returns without further processing.
        If the start time is valid, it sets the selected image capture time using 'set_selected_image_time' function, displays
        the remaining time for video export using 'get_remaining_export_time', and prints an end message using 'print_end_msg'.

        Args:
            change: The event object representing the button press.

        Returns:
            None
        """
        start_date = image_start_date.value
        start_hour = image_start_hours.value
        start_minute = image_start_minutes.value
        start_second = image_start_seconds.value

        # ISO format for start time
        # '2018-12-31T05:00:30.001000+09:00'
        start_datetime_iso = '{}T{:0=2}:{:0=2}:{:0=2}.000000+09:00'.format(
            start_date, start_hour, start_minute, start_second)
        start_datetime = datetime.fromisoformat(start_datetime_iso)

        # Future time cannot be specified.
        jst_now = datetime.now(JST)
        if jst_now.timestamp() < start_datetime.timestamp() or jst_now.timestamp() < start_datetime.timestamp():
            print()
            print('➕➕➕ ⛔ A future date and time has been specified. Please confirm the date and time again. ➕➕➕')
            return

        # Holds the entered value
        set_selected_image_time(start_datetime)

        # Display remaining time for video export
        get_remaining_export_time(soracom_api_client, camera_id)

        # Process end message display
        print_end_msg()

    # Handler Setup
    image_time_setting_button.on_click(on_image_time_setting_button_handler)


#
# --------------- Execution ---------------
#


# Process start message display
print_start_msg()

# Get the SORACOM API client.
soracom_api_client = get_soracom_api_client()

# Get the selected camera id.
selected_camera_id = get_selected_camera_id()

# Show the UI for specifying the date and time to download a still image.
show_image_time_widgets(soracom_api_client, selected_camera_id)

# Process end message display
print_end_msg()


In [ ]:
# @title [付録] ステップ 7: 指定した開始時刻で静止画をダウンロードする

#
# --------------- Definition ---------------
#

# @markdown # Fill in the required information and run.
# @markdown - - -

# @markdown Using a **wide-angle correction** filter.
use_wide_angle_correction = False #@param {type:"boolean"}

# @markdown - - -

import time
from pprint import pprint


# Download a images within a designated range.
def download_images_time_range(soracom_api_client: APIClient, camera_id: str, image_time_list: list, image_out_path: str, use_wide_angle_correction: bool) -> tuple[list, list]:
    """
    Download images within a designated time range.

    This function downloads images within a designated time range. It accepts an SORACOM API client, camera ID, a list of image times,
    an output path for the downloaded images, and a flag indicating whether to use wide-angle correction. It goes through each image
    in the time list, requests the image export, waits for it to become available, and then downloads the images. The progress is displayed
    with progress bars.

    Args:
        soracom_api_client (APIClient): The SORACOM API client for interacting with the SORACOM platform.
        camera_id (str): The ID of the camera associated with the image acquisition.
        image_time_list (list): A list of datetime objects representing the times of the images to be downloaded.
        image_out_path (str): The path where the downloaded images will be saved.
        use_wide_angle_correction (bool): A flag indicating whether wide-angle correction should be used during image export.

    Returns:
        tuple[list, list]: A tuple containing two lists: the first list contains information about successfully downloaded images,
        and the second list contains information about images that failed to download.

    Note:
        - The function uses helper functions like 'wait_for_download' to display waiting progress and 'get_export_image' to download images.
        - The function handles various export status conditions, such as 'completed', 'failed', 'limitExceeded', and 'expired'.
        - Progress information is displayed as the function processes each image.
        - The 'WAIT_TIME_LOWER_LIMIT' constant is used for the minimum waiting time.
    """

    # Display the progress bar and wait for the specified time
    def wait_for_download(wait_sec: int) -> bool:
        """
        Display a progress bar and wait for the specified time.

        Args:
            wait_sec (int): The number of seconds to wait.

        Returns:
            bool: True if the waiting is completed, False otherwise.
        """

        print()
        print('# Waiting time estimate = ', wait_sec, '(seconds)')

        progress = widgets.IntProgress(
            value=0,
            min=0,
            max=wait_sec,
            description='Waiting:',
            bar_style='info',  # 'success', 'info', 'warning', 'danger' or ''
            # style={'bar_color': 'maroon'},
            orientation='horizontal'
        )

        progress_text = widgets.Label(value="0 %")
        display(widgets.HBox([progress, progress_text]))

        start = datetime.now(JST)
        for i in range(wait_sec):
            per = ((i + 1) / wait_sec) * 100
            progress_text.value = str(int(per)) + ' %'
            progress.value = i + 1
            time.sleep(1)
        end = datetime.now(JST)

        print('# Actual waiting time = ', (end - start).total_seconds(), '(seconds)')

        return True

    # Get image file from the export URL
    def get_export_image(url: str, image_path: str) -> dict | None:
        """
        Download an image from a specified export URL and save it to a local file.

        This function is responsible for downloading an image from a given export URL and saving it to a local file specified by the 'image_path' parameter.
        It also performs error handling and returns information about any failures during the download process.

        Args:
            url (str): The URL from which to download the image.
            image_path (str): The local file path where the downloaded image will be saved.

        Returns:
            dict or None: If the download process encounters an error, it returns a dictionary containing error information.
                          If the download is successful, it returns None to indicate success.

        Note:
            - The function uses 'send_get_content' to request and retrieve the image content from the specified URL.
            - It checks for success using 'contents_result.check_success()' and handles failures accordingly.
            - If successful, it saves the image content to the specified 'image_path'.
            - Error information is returned in the form of a dictionary, which can be used for further analysis or logging.
        """
        print()
        print('# 🎨 Download Image')
        print('# url = ', url)
        print('# image_path = ', image_path)

        # Request to retrieve content
        contents_result = send_get_content(url)

        # failure
        if not contents_result.check_success():
            return contents_result.additions

        # Save as a image file
        with open(image_path, "wb") as f:
            f.write(contents_result.content)

        return None


    # Define fixed values for waiting time
    WAIT_TIME_LOWER_LIMIT: Final = 3

    # List of successes and failures
    success_list = list()
    failure_list = list()

    print()
    print('# 🖼️ Start downloading the images.')
    print('# Total downloads = ', len(image_time_list))

    for index, item in enumerate(image_time_list):
        print()
        print('# ℹ️ Current entry = {}/{}'.format(index+1, len(image_time_list)))
        print('# time = ', item)

        # Record execution status
        state_records = list()
        state_records.append(item)

        # Request image export
        requested = None
        api_result = soracom_api_client.export_sora_cam_recorded_image(camera_id, int(item.timestamp() * 1000), use_wide_angle_correction)

        state_records.append(vars(api_result))

        if not api_result.check_success():
            print()
            print('# 🚫 API request to export image failed.')
            # Errors are logged and the next task is executed
            failure_list.append(state_records)
            print('# 🔝 Processing for loop continues')
            continue

        requested = api_result.response
        print()
        print('# 🏮 Export Requested = ', api_result.get_display_response())

        # Must wait for it to be available for download
        # For still images, you can download them after waiting a few seconds, so we use a fixed value.
        wait_sec = WAIT_TIME_LOWER_LIMIT

        # Wait until you can download it.
        export_status = None
        while True:
            # Display progress to prevent the system from being detected as stopped after a long period of inactivity.
            if wait_for_download(wait_sec):
                # Check to see if it is available for download
                api_result = soracom_api_client.get_sora_cam_exported_image(camera_id, requested['exportId'])

                state_records.append(vars(api_result))

                if not api_result.check_success():
                    print()
                    print('# 🚫 API request to check export status failed.')
                    # Errors are logged and the next task is executed
                    failure_list.append(state_records)
                    print('# 🔝 Processing while loop break')
                    break

                export_status = api_result.response
                print()
                print('# 🔍 Export Status = ', api_result.get_display_response())

                # Download available only if 'status' is 'completed'
                # [ initializing, processing, completed, retrying, failed, limitExceeded, expired ]
                # Reference: https://users.soracom.io/ja-jp/tools/api/reference/#/SoraCam/listSoraCamDeviceVideoExports
                if export_status['status'] == 'completed':
                    print()
                    print('# ✅ The image is now available for download.')

                    # Perform the actual download.
                    image_url = export_status['url']
                    image_file_path = os.path.join(image_out_path, extract_filename_from_URL(image_url))

                    image_result = get_export_image(image_url, image_file_path)

                    if image_result is not None:
                        print()
                        print('# 🚫 Image download request failed.')
                        # Errors are logged and the next task is executed
                        state_records.append(image_result)
                        failure_list.append(state_records)
                        print('# 🔝 Processing while loop break')
                        break

                    # If you are successful up to the download, record it and move on to the next task.
                    success_list.append(state_records)
                    print()
                    print('# 💮 It all worked. while loop break')
                    break

                # For obvious errors
                if export_status['status'] == 'failed' or export_status['status'] == 'limitExceeded' or export_status['status'] == 'expired':
                    print()
                    print('# 🚫 Export status is an obvious failure.')
                    # Errors are logged and the next task is executed
                    failure_list.append(state_records)
                    print('# 🔝 Processing while loop break')
                    break

                # If it does not download, wait again
                wait_sec = int(wait_sec * 0.25)
                wait_sec = WAIT_TIME_LOWER_LIMIT if wait_sec <= WAIT_TIME_LOWER_LIMIT else wait_sec

                print()
                print('# 🔁 Wait again = ', datetime.now(JST))

    return success_list, failure_list


#
# --------------- Execution ---------------
#


# Process start message display
print_start_msg()

# Get the SORACOM API client.
soracom_api_client = get_soracom_api_client()

# Get the selected camera id.
selected_camera_id = get_selected_camera_id()

# Get the selected image time.
selected_image_time = get_selected_image_time()

# Initialize directory to store download files
init_dir(EXPORT_IMAGES_DIR)

# Download Image
success_list, failure_list = download_images_time_range(soracom_api_client, selected_camera_id, [selected_image_time], EXPORT_IMAGES_DIR, use_wide_angle_correction)

# Displays successful and unsuccessful attempts, respectively.
if len(failure_list) > 0:
    print()
    print('➕➕➕ ⛔ Some errors have been found during execution. Please check the content. ➕➕➕')
    print()
    print('# failure_count = ', len(failure_list))
    pprint(failure_list)

    if len(success_list) > 0:
        print()
        print('# 💬 Some of them are successful. Please check the contents.')
        print()
        print('# success_count = ', len(success_list))
        pprint(success_list)

# Process end message display
print_end_msg()


In [ ]:
# @title [付録] ステップ 8: ダウンロードした静止画を表示して確認する


#
# --------------- Execution ---------------
#


# Process start message display
print_start_msg()

# Display images in a grid
display_grid_images(EXPORT_IMAGES_DIR, '*.jpg')

# Process end message display
print_end_msg()

In [ ]:
# @title [付録] ステップ 9: ダウンロードした静止画をコピーする


#
# --------------- Execution ---------------
#


# Process start message display
print_start_msg()

# Source directory
source_dir = EXPORT_IMAGES_DIR

# Destination directory
destination_dir = EVENT_IMAGE_DIR

# Copy from the location where the still image is saved to the location where event images are saved.
copy_directory(source_dir, destination_dir)

# Process end message display
print_end_msg()
